# Orbits of bitangents on symmetric quartics

## Table of contents
1. [Introduction](#introduction)
2. [Functions](#functions)
3. [Type I: The Klein ($\text{PSL}_2(7)$) quartic](#klein)
4. [Type II: The Clebsch](#fermat)
5. [Type III: The (48,33) quartic](#type3)
6. [Type IV: The $S_4$ quartic](#S4)
7. [Type V: The (16,13) quartic](#1613)
8. [Type VI: The $C_9$ quartic](#C9)
9. [Type VII: The $D_4$ quartic](#D4)
10. [Type IX: The $S_3$ quartic](#S3)
11. [Type X: The $K_4$ quartic](#K4)
12. [Type XI: The $C_3$ quartic](#C3)
13. [Type XII: The $C_2$ quartic](#C2)

# Introduction<a name="introduction"></a>
The following code is taken from [Vinzant's webpage](http://sites.math.washington.edu/~vinzant/research/quartics/quartictype.sage) from [her research on quartics](http://sites.math.washington.edu/~vinzant/research/quartics.html) with Plaumann and Sturmfels. It inputs a quartic polynomial and outputs (among other things) equations for its tritangents.

# Functions<a name="functions"></a>

In [1]:
#INPUT: A homogeneous quartic in QQ[x,y,z]
#OUTPUT: The type of the real variety (from Table 1) and the 28 bitangents (with floating point coefficients)
F=QQ; T.<x,y,z>=PolynomialRing(F)

#INPUT:
# f=10*x^4+15*x^3*y-17*x^2*y^2+15*x*y^3+10*y^4+15*x^3*z-833*x^2*y*z-833*x*y^2*z+15*y^3*z-17*x^2*z^2-833*x*y*z^2-17*y^2*z^2 + 15*x*z^3 + 15*y*z^3 + 10*z^4
# f = z^3*y + x*(x-y)*(x-2*y)*(x-3*y)

def compute_bitangents(f):
    F=QQ; T.<x,y,z>=PolynomialRing(F)
    # Check whether the quartic is smooth
    Grad=ideal(f,diff(f,x),diff(f,y),diff(f,z))
    # if not Grad.dimension()==0:
    #     sys.exit("Quartic is not smooth!")
        
    R.<x,y,z,a,b,a0,a1,a2,a3,a4>=PolynomialRing(F)
    f0=f.base_extend(R)
    S.<a,b>=PolynomialRing(F)
    digits=50
    threshold=0.000000000001
    almostzero=threshold
    
    Line= a*x+b*y+z;
    puresquare=ideal(a0*a3^2-a1^2*a4,8*a0^2*a3-4*a0*a1*a2+a1^3,8*a1*a4^2-4*a2*a3*a4+a3^3,8*a0*a1*a4-4*a0*a2*a3+a1^2*a3,8*a0*a3*a4-4*a1*a2*a4+a1*a3^2,16*a0^2*a4+2*a0*a1*a3-4*a0*a2^2+a1^2*a2,16*a0*a4^2+2*a1*a3*a4-4*a2^2*a4+a2*a3^2);
    Res=f0.resultant(Line,z)    
    Res=Res.subs(y=1)    
    phi=hom(R,S,[0,0,0,a,b,Res.coefficient({x:0}),Res.coefficient({x:1}),Res.coefficient({x:2}),Res.coefficient({x:3}),Res.coefficient({x:4})])
    bit1 = phi(puresquare)     
    
    I_ideal=singular.groebner(singular(bit1))
    singular.lib('solve.lib')
    VRing=singular.solve(I_ideal,digits)
    singular.set_ring(VRing)
    B1=singular("SOL")
    
    nreal1=0
    Bitangents=[]
    RealBitangents=[]
    for k in [1..len(B1)]:
        real=0;
        if ((B1[k][1].impart()).absValue()<threshold) and ((B1[k][2].impart()).absValue()<threshold): 
            real=1
            RealBitangents=RealBitangents+[(float(B1[k][1].repart())+float(B1[k][1].impart())*i)*x+(float(B1[k][2].repart())+float(B1[k][2].impart())*i)*y+z]
        nreal1=nreal1+real
        Bitangents=Bitangents+[(float(B1[k][1].repart())+float(B1[k][1].impart())*i)*x+(float(B1[k][2].repart())+float(B1[k][2].impart())*i)*y+z]
    Line=a*x+y     
    Res=f0.resultant(Line,y)    
    Res=Res.subs(z=1)   
    phi=hom(R,S,[0,0,0,a,0,Res.coefficient({x:0}),Res.coefficient({x:1}),Res.coefficient({x:2}),Res.coefficient({x:3}),Res.coefficient({x:4})])  
    bit2=phi(puresquare)+ideal(b)
    
    if dimension(bit2)==-1: nreal2=0
    else: 
          I_ideal=singular.groebner(singular(bit2))
          singular.lib('solve.lib')
          VRing=singular.solve(I_ideal,digits)
          singular.set_ring(VRing)
          B2=singular("SOL")
          nreal2=0
          for k in [1..len(B2)]:
                real=0
                if ((B2[k][1].impart()).absValue()<threshold) and ((B2[k][2].impart()).absValue()<threshold):
                    real=1
                    RealBitangents=RealBitangents+[(float(B2[k][1].repart())+float(B2[k][1].impart())*i)*x+y]
                nreal2=nreal2+real  
                Bitangents=Bitangents+[(float(B2[k][1].repart())+float(B2[k][1].impart())*i)*x+y]
    
    Res=f0.resultant(x)
    Res=Res.subs(z=1)
    phi=hom(R,F,[0,0,0,0,0,Res.coefficient({y:0}),Res.coefficient({y:1}),Res.coefficient({y:2}),Res.coefficient({y:3}),Res.coefficient({y:4})])  
    bit3=phi(puresquare)
    if bit3==ideal(0):
        nreal3=1
        Bitangents=Bitangents+[x]
        RealBitangents=RealBitangents+[x]
    else: nreal3=0
    
    NRealBit=nreal1+nreal2+nreal3
    if len(Bitangents)!=28:
        return OSError("Something has gone wrong. We found "+str(len(Bitangents))+" bitangents")
    print("The quartic has 28 bitangets, stored in 'Bitangents', and "+str(NRealBit)+" real bitangents, stored in 'RealBitangents'.")
    
    return Bitangents

In [2]:
from sage.symbolic.expression_conversions import polynomial

# Round complex numbers
def round_cx(z0,n):
    return float(round(z0.real(),n)) + float(round(z0.imag(),n))*I

def coeffs_to_line(list_of_coeffs):
    return polynomial(x*list_of_coeffs[0] + y*list_of_coeffs[1] + z*list_of_coeffs[2],CC)

# Provide a normalized version of the bitangent
def normalize(bitangent):
    b = bitangent.change_ring(ComplexField(30))
    c = b*(1/b.coefficients()[0])
    if len(c.coefficients())==3:
        new_coeffs = [round_cx(a,5) for a in c.coefficients()]
        c = new_coeffs[0]*x + new_coeffs[1]*y + new_coeffs[2]*z
    return c


def gp_action(M,B):
    vec = matrix([[B.coefficient({x:1})],[B.coefficient({y:1})],[B.coefficient({z:1})]])
    output_mat = (M*vec).transpose()
    w = list(list(output_mat)[0])
    outputline = w[0]*x + w[1]*y + w[2]*z
    return normalize(outputline)
    # return w

def extract_orbit(Bitangents,list_of_indices):
    orbit = [Bitangents[j] for j in list_of_indices]
    remaining_Bitangents = [b for b in Bitangents if b not in orbit]
    return orbit,remaining_Bitangents

We write some functions to make the equations of bitangents more readable and act on them via a subgroup $G\le \text{PGL}_3(\mathbb{C})$.

In [3]:
# We also add some commands we'll use throughout
def matrix_to_list(M):
    return [list(r) for r in M.rows()]

In [4]:
# Generates a candidate list of the orbit of an input bitangent under a list of group elements
def candidate_orbit(bitangent,list_of_gp_elements):
    output_list = [bitangent]
    isotropy_gens = []
    for M in list_of_gp_elements:
        output_list.append(gp_action(M,bitangent))
        if gp_action(M,bitangent) == bitangent:
            isotropy_gens.append(M)
    for line in output_list:
        print(line)
    print('I think isotropy is generated by:\n')
    for isotropy_elt in isotropy_gens:
        print(isotropy_elt)

In [5]:
def investigate_orbit(bitangent,group_dictionary):
    output_list = [bitangent]
    isotropy_gens = []
    # Iterate over the group
    for g in group_dictionary.keys():        
        output_list.append(gp_action(group_dictionary[g],bitangent))
        if gp_action(group_dictionary[g],bitangent) == bitangent:
            isotropy_gens.append(g)
    for line in set(output_list):
        print(line)
    print('I think isotropy is generated by:\n')
    for isotropy_elt in isotropy_gens:
        print(isotropy_elt)

In [6]:
def trim(bitangent):
    x_coeff = CC(bitangent.coefficient({x:1}));
    y_coeff = CC(bitangent.coefficient({y:1}));
    z_coeff = CC(bitangent.coefficient({z:1}));
    return round_cx(x_coeff,3)*x + round_cx(y_coeff,3)*y + round_cx(z_coeff,3)*z

def investigate_orbit_Klein(bitangent,group_dictionary):
    output_list = [bitangent]
    isotropy_gens = []
    # Iterate over the group
    for g in group_dictionary.keys():        
        output_list.append(trim(gp_action(group_dictionary[g],bitangent)))
        if trim(gp_action(group_dictionary[g],bitangent)) == trim(bitangent):
            isotropy_gens.append([g,group_dictionary[g]])

    # Output the orbits
    for line in set(output_list):
        print(line)

    # Output the isotropy
    print('I think isotropy is generated by:\n')
    for isotropy_elt in isotropy_gens:
        print(isotropy_elt)

In [7]:
############

def investigate_orbit_V(bitangent,group_dictionary):
    output_list = [bitangent]
    isotropy_gens = []
    # Iterate over the group
    for g in group_dictionary.keys():        
        output_list.append(gp_action(group_dictionary[g],bitangent))
        if gp_action(group_dictionary[g],bitangent) == bitangent:
            isotropy_gens.append([g,group_dictionary[g]])

    # Output the orbits
    for line in set(output_list):
        print(line)

    # Output the isotropy
    print('I think isotropy is generated by:\n')
    for isotropy_elt in isotropy_gens:
        print(isotropy_elt)
        # list_version = matrix_to_list(isotropy_elt)
        # # print(list_version)
        # output_list = []
        # for row in list_version:
        #     new_row = []
        #     for j in row:
        #         # every entry in the matx representation is either totally real or totally imaginary
        #         if j.real != 0:
        #             new_row.append(int(j.real()))
        #         elif j.imag != 0:
        #             new_row.append(int(j.imag())*I)
        #         else:
        #             print('error')
        #     output_list.append(new_row)
        # print(str(output_list) + ',')

# def investigate_orbit2(bitangent,group_dictionary):
#     output_list = [bitangent]
#     isotropy_gens = []
#     # Iterate over the group
#     for g in group_dictionary.keys():        
#         output_list.append(gp_action(group_dictionary[g],bitangent))
#         if gp_action(group_dictionary[g],bitangent) == bitangent:
#             isotropy_gens.append(group_dictionary[g])
#     for line in set(output_list):
#         print(line)
#     print('I think isotropy is generated by:\n')
#     for isotropy_elt in isotropy_gens:
#         print(isotropy_elt)

In [8]:
def investigate_orbit_S4(bitangent,group_dictionary):
    output_list = [bitangent]
    isotropy_gens = []
    # Iterate over the group
    for g in group_dictionary.keys():        
        output_list.append(gp_action(group_dictionary[g],bitangent))
        if gp_action(group_dictionary[g],bitangent) == bitangent:
            isotropy_gens.append(group_dictionary[g])
    for line in set(output_list):
        print(line)
    print('I think isotropy is generated by:\n')
    for isotropy_elt in isotropy_gens:
        list_version = matrix_to_list(isotropy_elt)
        output_list = []
        for row in list_version:
            new_row = []
            for j in row:
                new_row.append(int(j.real()))
            output_list.append(new_row)
        print(str(output_list) + ',')

We want to compute all the orbits of the bitangents for the [cubic surfaces found here](https://mat.uab.cat/~francesc/mates/autgen3.pdf).

# Type I: The Klein quartic<a name="klein"></a>
We have an $G$-symmetric quartic with equation
$$\ldots$$
This is Type I in Dolgachev. and symmetries ...

In [9]:
# Klein = x^3*y + y^3*z + z^3*x;

# Klein_Bitangents = compute_bitangents(Klein)
# Klein_Bitangents = [normalize(b) for b in Klein_Bitangents]

In [10]:
# E7 = exp(2*pi*I/7)

In [11]:
# PSL2F7 = [ [ [ 0, 0, 1 ], [ 1, 0, 0 ], [ 0, 1, 0 ] ], [ [ 0, 0, E7^6 ], [ E7^3, 0, 0 ], [ 0, E7^5, 0 ] ], [ [ 0, 0, E7^5 ], [ E7^6, 0, 0 ], [ 0, E7^3, 0 ] ], [ [ 0, 0, E7^4 ], [ E7^2, 0, 0 ], [ 0, E7, 0 ] ], [ [ 0, 0, E7^3 ], [ E7^5, 0, 0 ], [ 0, E7^6, 0 ] ], [ [ 0, 0, E7^2 ], [ E7, 0, 0 ], [ 0, E7^4, 0 ] ], [ [ 0, 0, E7 ], [ E7^4, 0, 0 ], [ 0, E7^2, 0 ] ], [ [ 0, 1, 0 ], [ 0, 0, 1 ], [ 1, 0, 0 ] ], [ [ 0, E7^6, 0 ], [ 0, 0, E7^3 ], [ E7^5, 0, 0 ] ], [ [ 0, E7^5, 0 ], [ 0, 0, E7^6 ], [ E7^3, 0, 0 ] ], [ [ 0, E7^4, 0 ], [ 0, 0, E7^2 ], [ E7, 0, 0 ] ], [ [ 0, E7^3, 0 ], [ 0, 0, E7^5 ], [ E7^6, 0, 0 ] ], [ [ 0, E7^2, 0 ], [ 0, 0, E7 ], [ E7^4, 0, 0 ] ], [ [ 0, E7, 0 ], [ 0, 0, E7^4 ], [ E7^2, 0, 0 ] ], [ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ] ], [ [ -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6 ], [ 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6 ], [ -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6 ] ], [ [ -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6 ], [ 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6 ], [ -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6 ] ], [ [ -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6 ], [ 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6 ], [ 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6 ] ], [ [ -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6 ], [ -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6 ], [ -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6 ] ], [ [ -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5 ], [ -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6 ], [ 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6 ] ], [ [ -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6 ], [ -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6 ], [ -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6 ] ], [ [ -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6 ], [ 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5 ], [ 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6 ] ], [ [ -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6 ], [ 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6 ], [ -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6 ] ], [ [ -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6 ], [ 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6 ], [ 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6 ] ], [ [ -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6 ], [ -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5 ], [ -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6 ] ], [ [ -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6 ], [ -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6 ], [ -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6 ] ], [ [ -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6 ], [ -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6 ], [ 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6 ] ], [ [ -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6 ], [ -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6 ], [ 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6 ] ], [ [ -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5 ], [ 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6 ], [ 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6 ] ], [ [ -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6 ], [ 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6 ], [ -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6 ] ], [ [ -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6 ], [ 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5 ], [ -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6 ] ], [ [ -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6 ], [ 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6 ], [ 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6 ] ], [ [ -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6 ], [ 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6 ], [ -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6 ] ], [ [ -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5 ], [ -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6 ], [ 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6 ] ], [ [ -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6 ], [ -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6 ], [ 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6 ] ], [ [ -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6 ], [ -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6 ], [ 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6 ] ], [ [ -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6 ], [ 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6 ], [ -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6 ] ], [ [ -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6 ], [ 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6 ], [ -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6 ] ], [ [ -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6 ], [ -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6 ], [ 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6 ] ], [ [ -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5 ], [ -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6 ], [ -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6 ] ], [ [ -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6 ], [ -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6 ], [ 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6 ] ], [ [ -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6 ], [ 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6 ], [ -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6 ] ], [ [ -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6 ], [ 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5 ], [ 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6 ] ], [ [ -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6 ], [ -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6 ], [ -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5 ] ], [ [ -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6 ], [ -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6 ], [ 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5 ] ], [ [ -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5 ], [ 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5 ], [ -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5 ] ], [ [ -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6 ], [ 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6 ], [ -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5 ] ], [ [ -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6 ], [ 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6 ], [ 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5 ] ], [ [ -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6 ], [ -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6 ], [ 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5 ] ], [ [ -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6 ], [ -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6 ], [ 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5 ] ], [ [ -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6 ], [ 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6 ], [ -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6 ] ], [ [ -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6 ], [ -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5 ], [ -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6 ] ], [ [ -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6 ], [ 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6 ], [ 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6 ] ], [ [ -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6 ], [ -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6 ], [ -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6 ] ], [ [ -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6 ], [ 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6 ], [ 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6 ] ], [ [ -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6 ], [ 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6 ], [ 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6 ] ], [ [ -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5 ], [ -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6 ], [ 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6 ] ], [ [ -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6 ], [ -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6 ], [ -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5 ] ], [ [ -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6 ], [ 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6 ], [ -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5 ] ], [ [ -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6 ], [ 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6 ], [ 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5 ] ], [ [ -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6 ], [ 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6 ], [ -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5 ] ], [ [ -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6 ], [ 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6 ], [ 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5 ] ], [ [ -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6 ], [ -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6 ], [ -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5 ] ], [ [ -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5 ], [ -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5 ], [ 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5 ] ], [ [ -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6 ], [ 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6 ], [ -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6 ] ], [ [ -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6 ], [ -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6 ], [ 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6 ] ], [ [ -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6 ], [ -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5 ], [ -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6 ] ], [ [ -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6 ], [ -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6 ], [ -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6 ] ], [ [ -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6 ], [ -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6 ], [ 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6 ] ], [ [ -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5 ], [ 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6 ], [ 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6 ] ], [ [ -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6 ], [ 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6 ], [ 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6 ] ], [ [ -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6 ], [ -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6 ], [ -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6 ] ], [ [ -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6 ], [ 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5 ], [ -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6 ] ], [ [ -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6 ], [ 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6 ], [ 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6 ] ], [ [ -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5 ], [ -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6 ], [ -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6 ] ], [ [ -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6 ], [ -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6 ], [ 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6 ] ], [ [ -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6 ], [ 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6 ], [ 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6 ] ], [ [ -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6 ], [ 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6 ], [ 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6 ] ], [ [ -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5 ], [ -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6 ], [ -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6 ] ], [ [ -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6 ], [ 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5 ], [ -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6 ] ], [ [ -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6 ], [ -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6 ], [ 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6 ] ], [ [ -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6 ], [ 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6 ], [ -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6 ] ], [ [ -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6 ], [ 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6 ], [ 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6 ] ], [ [ -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6 ], [ 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6 ], [ 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6 ] ], [ [ -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6 ], [ -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6 ], [ 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6 ] ], [ [ E7^6, 0, 0 ], [ 0, E7^3, 0 ], [ 0, 0, E7^5 ] ], [ [ E7^5, 0, 0 ], [ 0, E7^6, 0 ], [ 0, 0, E7^3 ] ], [ [ E7^4, 0, 0 ], [ 0, E7^2, 0 ], [ 0, 0, E7 ] ], [ [ E7^3, 0, 0 ], [ 0, E7^5, 0 ], [ 0, 0, E7^6 ] ], [ [ 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6 ], [ 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6 ], [ -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6 ] ], [ [ 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6 ], [ -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6 ], [ -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6 ] ], [ [ 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5 ], [ -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6 ], [ 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6 ] ], [ [ 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6 ], [ -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6 ], [ -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6 ] ], [ [ 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6 ], [ 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6 ], [ 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6 ] ], [ [ 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6 ], [ 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6 ], [ -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6 ] ], [ [ 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6 ], [ 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5 ], [ 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6 ] ], [ [ 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6 ], [ -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6 ], [ -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6 ] ], [ [ 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5 ], [ 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6 ], [ 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6 ] ], [ [ 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6 ], [ 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5 ], [ -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6 ] ], [ [ 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6 ], [ 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6 ], [ -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6 ] ], [ [ 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6 ], [ -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6 ], [ 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6 ] ], [ [ 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6 ], [ -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6 ], [ 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6 ] ], [ [ 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6 ], [ -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6 ], [ 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6 ] ], [ [ E7^2, 0, 0 ], [ 0, E7, 0 ], [ 0, 0, E7^4 ] ], [ [ 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6 ], [ 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6 ], [ -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6 ] ], [ [ 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6 ], [ 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5 ], [ -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6 ] ], [ [ 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5 ], [ -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6 ], [ 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6 ] ], [ [ 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6 ], [ -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6 ], [ -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6 ] ], [ [ 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6 ], [ 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6 ], [ 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6 ] ], [ [ 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6 ], [ -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6 ], [ 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6 ] ], [ [ 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6 ], [ 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6 ], [ 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6 ] ], [ [ 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6 ], [ 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6 ], [ -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6 ] ], [ [ 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6 ], [ 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6 ], [ -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6 ] ], [ [ 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6 ], [ 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6 ], [ 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6 ] ], [ [ 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6 ], [ 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6 ], [ -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6 ] ], [ [ 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6 ], [ -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6 ], [ 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6 ] ], [ [ 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5 ], [ -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6 ], [ -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6 ] ], [ [ 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6 ], [ -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5 ], [ 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6 ] ], [ [ 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6 ], [ -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6 ], [ -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6 ] ], [ [ 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6 ], [ -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6 ], [ 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6 ] ], [ [ 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6 ], [ -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5 ], [ -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6 ] ], [ [ 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6 ], [ -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6 ], [ -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6 ] ], [ [ 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5 ], [ 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6 ], [ 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6 ] ], [ [ 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6 ], [ 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6 ], [ 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6 ] ], [ [ 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6 ], [ 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6 ], [ 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6 ] ], [ [ 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6 ], [ -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6 ], [ -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6 ] ], [ [ 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5 ], [ -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6 ], [ -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6 ] ], [ [ 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6 ], [ -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6 ], [ 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6 ] ], [ [ 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6 ], [ 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6 ], [ -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6 ] ], [ [ 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6 ], [ 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6 ], [ 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6 ] ], [ [ 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6 ], [ 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6 ], [ -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6 ] ], [ [ 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6 ], [ 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5 ], [ 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6 ] ], [ [ 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5 ], [ 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6 ], [ -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6 ] ], [ [ 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6 ], [ 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6 ], [ 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6 ] ], [ [ 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6 ], [ 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5 ], [ -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6 ] ], [ [ 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6 ], [ -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6 ], [ -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6 ] ], [ [ 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6 ], [ -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6 ], [ 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6 ] ], [ [ 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6 ], [ -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6 ], [ 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6 ] ], [ [ 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6 ], [ -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6 ], [ 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6 ] ], [ [ 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6 ], [ -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6 ], [ -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6 ] ], [ [ 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6 ], [ -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5 ], [ -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6 ] ], [ [ 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6 ], [ 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6 ], [ 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6 ] ], [ [ 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6 ], [ 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6 ], [ -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6 ] ], [ [ 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6 ], [ 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6 ], [ 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6 ] ], [ [ 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5 ], [ -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6 ], [ 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6 ] ], [ [ 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6 ], [ 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6 ], [ 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6 ] ], [ [ 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6 ], [ -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6 ], [ -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6 ] ], [ [ 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6 ], [ -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6 ], [ 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6 ] ], [ [ 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6 ], [ -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5 ], [ -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6 ] ], [ [ 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5 ], [ 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6 ], [ -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6 ] ], [ [ 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6 ], [ 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6 ], [ 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6 ] ], [ [ 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6 ], [ 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6 ], [ 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6 ] ], [ [ 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6 ], [ -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6 ], [ 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6 ] ], [ [ 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6 ], [ 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6 ], [ -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5 ] ], [ [ 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5 ], [ -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5 ], [ -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5 ] ], [ [ 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6 ], [ -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6 ], [ 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5 ] ], [ [ 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6 ], [ 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6 ], [ -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5 ] ], [ [ 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6 ], [ -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6 ], [ 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5 ] ], [ [ 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6 ], [ 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6 ], [ 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5 ] ], [ [ 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6 ], [ 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6 ], [ 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5 ] ], [ [ 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5 ], [ -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6 ], [ -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6 ] ], [ [ 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, -2/7*E7-2/7*E7^2-3/7*E7^4-4/7*E7^5-3/7*E7^6, -3/7*E7^2-2/7*E7^3-4/7*E7^4-2/7*E7^5-3/7*E7^6 ], [ -3/7*E7-4/7*E7^2-3/7*E7^3-2/7*E7^5-2/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, -2/7*E7-3/7*E7^2-3/7*E7^3-2/7*E7^4-4/7*E7^6 ], [ -3/7*E7-2/7*E7^2-4/7*E7^3-2/7*E7^4-3/7*E7^5, -4/7*E7-2/7*E7^3-3/7*E7^4-3/7*E7^5-2/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6 ] ], [ [ 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6 ], [ 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6 ], [ 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6 ] ], [ [ 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6 ], [ 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6 ], [ -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6 ] ], [ [ 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6 ], [ 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6 ], [ 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6 ] ], [ [ 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 2/7*E7-1/7*E7^2-2/7*E7^3-1/7*E7^4+2/7*E7^5, -2/7*E7-1/7*E7^3+2/7*E7^4+2/7*E7^5-1/7*E7^6 ], [ 2/7*E7^2-1/7*E7^3-2/7*E7^4-1/7*E7^5+2/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, -1/7*E7-1/7*E7^2+2/7*E7^4-2/7*E7^5+2/7*E7^6 ], [ -1/7*E7+2/7*E7^2+2/7*E7^3-1/7*E7^4-2/7*E7^6, 2/7*E7-2/7*E7^2+2/7*E7^3-1/7*E7^5-1/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6 ] ], [ [ 4/7*E7+2/7*E7^2+1/7*E7^3+1/7*E7^4+2/7*E7^5+4/7*E7^6, 3/7*E7+1/7*E7^2+1/7*E7^3+3/7*E7^4-1/7*E7^6, 1/7*E7-1/7*E7^2+1/7*E7^3+3/7*E7^5+3/7*E7^6 ], [ -1/7*E7+3/7*E7^3+1/7*E7^4+1/7*E7^5+3/7*E7^6, 2/7*E7+1/7*E7^2+4/7*E7^3+4/7*E7^4+1/7*E7^5+2/7*E7^6, 1/7*E7+3/7*E7^2-1/7*E7^3+3/7*E7^4+1/7*E7^5 ], [ 3/7*E7+3/7*E7^2+1/7*E7^4-1/7*E7^5+1/7*E7^6, 1/7*E7^2+3/7*E7^3-1/7*E7^4+3/7*E7^5+1/7*E7^6, 1/7*E7+4/7*E7^2+2/7*E7^3+2/7*E7^4+4/7*E7^5+1/7*E7^6 ] ], [ [ E7, 0, 0 ], [ 0, E7^4, 0 ], [ 0, 0, E7^2 ] ] ]

# keys = ['r' + str(j) for j in range(0,168)]
# PSL_dict = {}
# for j in range(0,168):
#     PSL_dict.update({keys[j]:matrix(CC,PSL2F7[j],immutable=True)});

In [12]:
# orbit_indices = [[13,4,5,6,2,1,26,3,27,14,15,0,12,19,16,24,25,18,17,21,20,11,10,22,7,8,23,9]]

# all_orbit_indices = flatten(orbit_indices)
# list_of_bitangent_orbits = []
# for o in orbit_indices:
#     orbit = []
#     for b_index in o:
#         orbit.append(Klein_Bitangents[b_index])
#     list_of_bitangent_orbits.append(orbit)

# print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
# print('Remaining bitangents:\n')
# for j in range(0,len(Klein_Bitangents)):
#     if not j in all_orbit_indices:
#         print(str(j) + ': ' + str(Klein_Bitangents[j]))

In [13]:
# investigate_orbit_Klein(Klein_Bitangents[0],PSL_dict)
# l1=Klein_Bitangents[0]
# x_coeff = l1.coefficient({x:1})
# CC(x_coeff).imag()
# trim(l1)

Checking this isotropy in GAP we get $[\text{PSL}_2(\mathbb{F}_7)/S_3]$.

# Type II: Fermat<a href="fermat"></a>

## (II) Bitangents

In [14]:
F=QQ; T.<x,y,z>=PolynomialRing(F)
Fermat = x^4 + y^4 + z^4

Fermat_Bitangents = compute_bitangents(Fermat)
Fermat_Bitangents = [normalize(b) for b in Fermat_Bitangents];Fermat_Bitangents

The quartic has 28 bitangets, stored in 'Bitangents', and 4 real bitangents, stored in 'RealBitangents'.


[y + (-0.70710678 - 0.70710678*I)*z,
 y + (-0.70710678 + 0.70710678*I)*z,
 y + (0.70710678 + 0.70710678*I)*z,
 y + (0.70710678 - 0.70710678*I)*z,
 x + y - z,
 x - y + z,
 x + 1.0*I*y + (-1.0*I)*z,
 x + (-1.0*I)*y + 1.0*I*z,
 x + (-0.70710678 - 0.70710678*I)*z,
 x + (-0.70710678 + 0.70710678*I)*z,
 x + (0.70710678 + 0.70710678*I)*z,
 x + (0.70710678 - 0.70710678*I)*z,
 x - y - z,
 x + y + z,
 x + (-1.0*I)*y + (-1.0*I)*z,
 x + 1.0*I*y + 1.0*I*z,
 x + 1.0*I*y - z,
 x + (-1.0*I)*y + z,
 x - y + (-1.0*I)*z,
 x + y + 1.0*I*z,
 x + (-1.0*I)*y - z,
 x + 1.0*I*y + z,
 x + y + (-1.0*I)*z,
 x - y + 1.0*I*z,
 x + (-0.70710678 - 0.70710678*I)*y,
 x + (-0.70710678 + 0.70710678*I)*y,
 x + (0.70710678 + 0.70710678*I)*y,
 x + (0.70710678 - 0.70710678*I)*y]

In [15]:
Fermat_Aut_Elts = [ [ [ -1, 0, 0 ], [ 0, -1, 0 ], [ 0, 0, 1 ] ],
  [ [ -1, 0, 0 ], [ 0, 0, -1 ], [ 0, 1, 0 ] ],
  [ [ -1, 0, 0 ], [ 0, 0, 1 ], [ 0, -1, 0 ] ],
  [ [ -1, 0, 0 ], [ 0, 0, -E(4) ], [ 0, -E(4), 0 ] ],
  [ [ -1, 0, 0 ], [ 0, 0, E(4) ], [ 0, E(4), 0 ] ],
  [ [ -1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, -1 ] ],
  [ [ -1, 0, 0 ], [ 0, -E(4), 0 ], [ 0, 0, -E(4) ] ],
  [ [ -1, 0, 0 ], [ 0, E(4), 0 ], [ 0, 0, E(4) ] ],
  [ [ 0, -1, 0 ], [ -1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, -1, 0 ], [ 0, 0, -1 ], [ 1, 0, 0 ] ],
  [ [ 0, -1, 0 ], [ 0, 0, 1 ], [ -1, 0, 0 ] ],
  [ [ 0, -1, 0 ], [ 0, 0, -E(4) ], [ -E(4), 0, 0 ] ],
  [ [ 0, -1, 0 ], [ 0, 0, E(4) ], [ E(4), 0, 0 ] ],
  [ [ 0, -1, 0 ], [ 1, 0, 0 ], [ 0, 0, -1 ] ],
  [ [ 0, -1, 0 ], [ -E(4), 0, 0 ], [ 0, 0, -E(4) ] ],
  [ [ 0, -1, 0 ], [ E(4), 0, 0 ], [ 0, 0, E(4) ] ],
  [ [ 0, 0, -1 ], [ -1, 0, 0 ], [ 0, 1, 0 ] ],
  [ [ 0, 0, -1 ], [ 0, -1, 0 ], [ 1, 0, 0 ] ],
  [ [ 0, 0, -1 ], [ 0, 1, 0 ], [ -1, 0, 0 ] ],
  [ [ 0, 0, -1 ], [ 0, -E(4), 0 ], [ -E(4), 0, 0 ] ],
  [ [ 0, 0, -1 ], [ 0, E(4), 0 ], [ E(4), 0, 0 ] ],
  [ [ 0, 0, -1 ], [ 1, 0, 0 ], [ 0, -1, 0 ] ],
  [ [ 0, 0, -1 ], [ -E(4), 0, 0 ], [ 0, -E(4), 0 ] ],
  [ [ 0, 0, -1 ], [ E(4), 0, 0 ], [ 0, E(4), 0 ] ],
  [ [ 0, 0, 1 ], [ -1, 0, 0 ], [ 0, -1, 0 ] ],
  [ [ 0, 0, 1 ], [ 0, -1, 0 ], [ -1, 0, 0 ] ],
  [ [ 0, 0, 1 ], [ 0, 1, 0 ], [ 1, 0, 0 ] ],
  [ [ 0, 0, 1 ], [ 0, -E(4), 0 ], [ E(4), 0, 0 ] ],
  [ [ 0, 0, 1 ], [ 0, E(4), 0 ], [ -E(4), 0, 0 ] ],
  [ [ 0, 0, 1 ], [ 1, 0, 0 ], [ 0, 1, 0 ] ],
  [ [ 0, 0, 1 ], [ -E(4), 0, 0 ], [ 0, E(4), 0 ] ],
  [ [ 0, 0, 1 ], [ E(4), 0, 0 ], [ 0, -E(4), 0 ] ],
  [ [ 0, 0, -E(4) ], [ -1, 0, 0 ], [ 0, -E(4), 0 ] ],
  [ [ 0, 0, -E(4) ], [ 0, -1, 0 ], [ -E(4), 0, 0 ] ],
  [ [ 0, 0, -E(4) ], [ 0, 1, 0 ], [ E(4), 0, 0 ] ],
  [ [ 0, 0, -E(4) ], [ 0, -E(4), 0 ], [ -1, 0, 0 ] ],
  [ [ 0, 0, -E(4) ], [ 0, E(4), 0 ], [ 1, 0, 0 ] ],
  [ [ 0, 0, -E(4) ], [ 1, 0, 0 ], [ 0, E(4), 0 ] ],
  [ [ 0, 0, -E(4) ], [ -E(4), 0, 0 ], [ 0, -1, 0 ] ],
  [ [ 0, 0, -E(4) ], [ E(4), 0, 0 ], [ 0, 1, 0 ] ],
  [ [ 0, 0, E(4) ], [ -1, 0, 0 ], [ 0, E(4), 0 ] ],
  [ [ 0, 0, E(4) ], [ 0, -1, 0 ], [ E(4), 0, 0 ] ],
  [ [ 0, 0, E(4) ], [ 0, 1, 0 ], [ -E(4), 0, 0 ] ],
  [ [ 0, 0, E(4) ], [ 0, -E(4), 0 ], [ 1, 0, 0 ] ],
  [ [ 0, 0, E(4) ], [ 0, E(4), 0 ], [ -1, 0, 0 ] ],
  [ [ 0, 0, E(4) ], [ 1, 0, 0 ], [ 0, -E(4), 0 ] ],
  [ [ 0, 0, E(4) ], [ -E(4), 0, 0 ], [ 0, 1, 0 ] ],
  [ [ 0, 0, E(4) ], [ E(4), 0, 0 ], [ 0, -1, 0 ] ],
  [ [ 0, 1, 0 ], [ -1, 0, 0 ], [ 0, 0, -1 ] ],
  [ [ 0, 1, 0 ], [ 0, 0, -1 ], [ -1, 0, 0 ] ],
  [ [ 0, 1, 0 ], [ 0, 0, 1 ], [ 1, 0, 0 ] ],
  [ [ 0, 1, 0 ], [ 0, 0, -E(4) ], [ E(4), 0, 0 ] ],
  [ [ 0, 1, 0 ], [ 0, 0, E(4) ], [ -E(4), 0, 0 ] ],
  [ [ 0, 1, 0 ], [ 1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, 1, 0 ], [ -E(4), 0, 0 ], [ 0, 0, E(4) ] ],
  [ [ 0, 1, 0 ], [ E(4), 0, 0 ], [ 0, 0, -E(4) ] ],
  [ [ 0, -E(4), 0 ], [ -1, 0, 0 ], [ 0, 0, -E(4) ] ],
  [ [ 0, -E(4), 0 ], [ 0, 0, -1 ], [ -E(4), 0, 0 ] ],
  [ [ 0, -E(4), 0 ], [ 0, 0, 1 ], [ E(4), 0, 0 ] ],
  [ [ 0, -E(4), 0 ], [ 0, 0, -E(4) ], [ -1, 0, 0 ] ],
  [ [ 0, -E(4), 0 ], [ 0, 0, E(4) ], [ 1, 0, 0 ] ],
  [ [ 0, -E(4), 0 ], [ 1, 0, 0 ], [ 0, 0, E(4) ] ],
  [ [ 0, -E(4), 0 ], [ -E(4), 0, 0 ], [ 0, 0, -1 ] ],
  [ [ 0, -E(4), 0 ], [ E(4), 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, E(4), 0 ], [ -1, 0, 0 ], [ 0, 0, E(4) ] ],
  [ [ 0, E(4), 0 ], [ 0, 0, -1 ], [ E(4), 0, 0 ] ],
  [ [ 0, E(4), 0 ], [ 0, 0, 1 ], [ -E(4), 0, 0 ] ],
  [ [ 0, E(4), 0 ], [ 0, 0, -E(4) ], [ 1, 0, 0 ] ],
  [ [ 0, E(4), 0 ], [ 0, 0, E(4) ], [ -1, 0, 0 ] ],
  [ [ 0, E(4), 0 ], [ 1, 0, 0 ], [ 0, 0, -E(4) ] ],
  [ [ 0, E(4), 0 ], [ -E(4), 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, E(4), 0 ], [ E(4), 0, 0 ], [ 0, 0, -1 ] ],
  [ [ 1, 0, 0 ], [ 0, -1, 0 ], [ 0, 0, -1 ] ],
  [ [ 1, 0, 0 ], [ 0, 0, -1 ], [ 0, -1, 0 ] ],
  [ [ 1, 0, 0 ], [ 0, 0, 1 ], [ 0, 1, 0 ] ],
  [ [ 1, 0, 0 ], [ 0, 0, -E(4) ], [ 0, E(4), 0 ] ],
  [ [ 1, 0, 0 ], [ 0, 0, E(4) ], [ 0, -E(4), 0 ] ],
  [ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ] ],
  [ [ 1, 0, 0 ], [ 0, -E(4), 0 ], [ 0, 0, E(4) ] ],
  [ [ 1, 0, 0 ], [ 0, E(4), 0 ], [ 0, 0, -E(4) ] ],
  [ [ -E(4), 0, 0 ], [ 0, -1, 0 ], [ 0, 0, -E(4) ] ],
  [ [ -E(4), 0, 0 ], [ 0, 0, -1 ], [ 0, -E(4), 0 ] ],
  [ [ -E(4), 0, 0 ], [ 0, 0, 1 ], [ 0, E(4), 0 ] ],
  [ [ -E(4), 0, 0 ], [ 0, 0, -E(4) ], [ 0, -1, 0 ] ],
  [ [ -E(4), 0, 0 ], [ 0, 0, E(4) ], [ 0, 1, 0 ] ],
  [ [ -E(4), 0, 0 ], [ 0, 1, 0 ], [ 0, 0, E(4) ] ],
  [ [ -E(4), 0, 0 ], [ 0, -E(4), 0 ], [ 0, 0, -1 ] ],
  [ [ -E(4), 0, 0 ], [ 0, E(4), 0 ], [ 0, 0, 1 ] ],
  [ [ E(4), 0, 0 ], [ 0, -1, 0 ], [ 0, 0, E(4) ] ],
  [ [ E(4), 0, 0 ], [ 0, 0, -1 ], [ 0, E(4), 0 ] ],
  [ [ E(4), 0, 0 ], [ 0, 0, 1 ], [ 0, -E(4), 0 ] ],
  [ [ E(4), 0, 0 ], [ 0, 0, -E(4) ], [ 0, 1, 0 ] ],
  [ [ E(4), 0, 0 ], [ 0, 0, E(4) ], [ 0, -1, 0 ] ],
  [ [ E(4), 0, 0 ], [ 0, 1, 0 ], [ 0, 0, -E(4) ] ],
  [ [ E(4), 0, 0 ], [ 0, -E(4), 0 ], [ 0, 0, 1 ] ],
  [ [ E(4), 0, 0 ], [ 0, E(4), 0 ], [ 0, 0, -1 ] ] ]

keys = ['r' + str(j) for j in range(0,96)]
Fermat_dict = {}
for j in range(0,96):
    Fermat_dict.update({keys[j]:matrix(CC,Fermat_Aut_Elts[j],immutable=True)});

In [16]:
C8orbit = [[0,1,2,3,24,25,26,27,8,9,10,11]]
C6orbit = [[16,6,20,21,15,12,14,17,18,7,4,5,23,22,13,19]]
orbit_indices = C8orbit+C6orbit

all_orbit_indices = flatten(orbit_indices)
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(Fermat_Bitangents[b_index])
    list_of_bitangent_orbits.append(orbit)

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(Fermat_Bitangents)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(Fermat_Bitangents[j]))

Orbits of bitangents: [[y + (-0.70710678 - 0.70710678*I)*z, y + (-0.70710678 + 0.70710678*I)*z, y + (0.70710678 + 0.70710678*I)*z, y + (0.70710678 - 0.70710678*I)*z, x + (-0.70710678 - 0.70710678*I)*y, x + (-0.70710678 + 0.70710678*I)*y, x + (0.70710678 + 0.70710678*I)*y, x + (0.70710678 - 0.70710678*I)*y, x + (-0.70710678 - 0.70710678*I)*z, x + (-0.70710678 + 0.70710678*I)*z, x + (0.70710678 + 0.70710678*I)*z, x + (0.70710678 - 0.70710678*I)*z], [x + 1.0*I*y - z, x + 1.0*I*y + (-1.0*I)*z, x + (-1.0*I)*y - z, x + 1.0*I*y + z, x + 1.0*I*y + 1.0*I*z, x - y - z, x + (-1.0*I)*y + (-1.0*I)*z, x + (-1.0*I)*y + z, x - y + (-1.0*I)*z, x + (-1.0*I)*y + 1.0*I*z, x + y - z, x - y + z, x - y + 1.0*I*z, x + y + (-1.0*I)*z, x + y + z, x + y + 1.0*I*z]]

Remaining bitangents:



In [17]:
investigate_orbit(Fermat_Bitangents[4], Fermat_dict)

x + y + 1.0*I*z
x + 1.0*I*y + 1.0*I*z
x - y + z
x - y + (-1.0*I)*z
x + (-1.0*I)*y + z
x + y + z
x - y - z
x + (-1.0*I)*y + (-1.0*I)*z
x + y + (-1.0*I)*z
x + (-1.0*I)*y - z
x + 1.0*I*y + (-1.0*I)*z
x + 1.0*I*y + z
x + y - z
x + 1.0*I*y - z
x - y + 1.0*I*z
x + (-1.0*I)*y + 1.0*I*z
I think isotropy is generated by:

r18
r21
r49
r53
r73
r77


## (II) Isotropy

In [18]:
Fermat_dict['r77']

[ 1.00000000000000 0.000000000000000 0.000000000000000]
[0.000000000000000  1.00000000000000 0.000000000000000]
[0.000000000000000 0.000000000000000  1.00000000000000]

In [19]:
Fr18 = [[0,0,-1],[0,1,0],[-1,0,0]];
Fr21 = [[0,0,-1],[1,0,0],[0,-1,0]];
Fr49 = [[0,1,0],[0,0,-1],[-1,0,0]];
Fr53 = [[0,1,0],[1,0,0],[0,0,1]];
Fr73 =[[1,0,0],[0,0,-1],[0,-1,0]];


In [20]:
Fr6 = [[-1,0,0],[0,-E(4),0],[0,0,-E(4)]];
Fr7 = [[-1,0,0],[0,E(4),0],[0,0,E(4)]];
Fr72 = [[1,0,0],[0,-1,0],[0,0,-1]];
Fr81 = [[-E(4),0,0],[0,0,-1],[0,-E(4),0]];
Fr82 = [[-E(4),0,0],[0,0,1],[0,E(4),0]];
Fr91 = [[E(4),0,0],[0,0,-E(4)],[0,1,0]];
Fr92 = [[E(4),0,0],[0,0,E(4)],[0,-1,0]];

# Type III: The $(48,33)$ quartic<a name="type3"></a>
We have an $G$-symmetric quartic with equation
$$x^4 + y^4 + z^4 + (4*rho+2)*x^2*y^2$$
This is Type III in Dolgachev. and symmetries ...

## (III) Bitangents

Dolgachev includes the quartic
$$x^4 + y^4 + z^4 + (4*rho+2)*(x^2)*(y^2)$$
on his table, and Henn does the same.

The problem is that $\rho\notin\mathbb{Q}$, so we rewrite the code to try to run over another field.


<!-- This is not smooth (todo check). Instead we use a quartic found in Theorem 16 of the Bars note -->

In [21]:
F=CyclotomicField(3);
T.<x,y,z>=PolynomialRing(F);T
R.<x,y,z,a,b,a0,a1,a2,a3,a4>=PolynomialRing(F)

In [22]:
#INPUT: A homogeneous quartic in QQ[x,y,z]
#OUTPUT: The type of the real variety (from Table 1) and the 28 bitangents (with floating point coefficients)
F=CyclotomicField(3); T.<x,y,z>=PolynomialRing(F)

#INPUT:
# f=10*x^4+15*x^3*y-17*x^2*y^2+15*x*y^3+10*y^4+15*x^3*z-833*x^2*y*z-833*x*y^2*z+15*y^3*z-17*x^2*z^2-833*x*y*z^2-17*y^2*z^2 + 15*x*z^3 + 15*y*z^3 + 10*z^4
# f = z^3*y + x*(x-y)*(x-2*y)*(x-3*y)



In [23]:
def compute_bitangents_Q3(f):
    F=CyclotomicField(3); T.<x,y,z>=PolynomialRing(F)
    # Check whether the quartic is smooth
    # Grad=ideal(f,diff(f,x),diff(f,y),diff(f,z))
    # if not Grad.dimension()==0:
    #     sys.exit("Quartic is not smooth!")
    f = T(f)
    R.<x,y,z,a,b,a0,a1,a2,a3,a4>=PolynomialRing(F)
    f0=f.base_extend(R)
    S.<a,b>=PolynomialRing(F)
    digits=50
    threshold=0.000000000001
    almostzero=threshold

    # Make a formal line
    Line= a*x+b*y+z;
    
    puresquare=ideal(a0*a3^2-a1^2*a4,8*a0^2*a3-4*a0*a1*a2+a1^3,8*a1*a4^2-4*a2*a3*a4+a3^3,8*a0*a1*a4-4*a0*a2*a3+a1^2*a3,8*a0*a3*a4-4*a1*a2*a4+a1*a3^2,16*a0^2*a4+2*a0*a1*a3-4*a0*a2^2+a1^2*a2,16*a0*a4^2+2*a1*a3*a4-4*a2^2*a4+a2*a3^2);
    Res=f0.resultant(Line,z)    
    Res=Res.subs(y=1)    
    phi=hom(R,S,[0,0,0,a,b,Res.coefficient({x:0}),Res.coefficient({x:1}),Res.coefficient({x:2}),Res.coefficient({x:3}),Res.coefficient({x:4})])
    bit1 = phi(puresquare)    
    
    I_ideal=singular.groebner(singular(bit1))
    return I_ideal
Dolg = x^4 + y^4 + z^4 + 2*sqrt(-3)*(x^2)*(y^2)
F=CyclotomicField(12); T.<x,y,z>=PolynomialRing(F)

# Grad = ideal(Dolg,diff(Dolg,x),diff(Dolg,y),diff(Dolg,z))
Iideal = compute_bitangents_Q3(Dolg);Iideal
# Grad.dimension()
# cyc

R.<a,b> =PolynomialRing(CC)
rho = exp(2*pi*i/3)
a,b = var('a b')
f0 = a^4+(-4*rho-2)*a^2*b^2+b^4+4
f1 = 3*a^3*b^3+(2*rho+1)*a*b^5+(4*rho+2)*a*b
f2 = 7*a^2*b^5+(2*rho+1)*b^7+2*a^2*b+(8*rho+4)*b^3
f3 = 2*a*b^7+(2*rho+1)*a^3*b+a*b^3
f4 = 2*b^9-5*a^4*b+(-6*rho-3)*a^2*b^3+10*b^5+8*b


candidates = solve([f1==0,f2==0,f3==0,f0==0,f4==0],a,b); candidates

[[a == -1/2*I*sqrt(3) - 1/2, b == -1/2*I*sqrt(3) - 1/2], [a == 1/2*I*sqrt(3) + 1/2, b == -1/2*I*sqrt(3) - 1/2], [a == -1/2*sqrt(3) - 1/2*I, b == 1/2*I*sqrt(3) - 1/2], [a == 1/2*sqrt(3) + 1/2*I, b == 1/2*I*sqrt(3) - 1/2], [a == -1/2*sqrt(3) - 1/2*I, b == -1/2*I*sqrt(3) + 1/2], [a == 1/2*sqrt(3) + 1/2*I, b == -1/2*I*sqrt(3) + 1/2], [a == -1/2*I*sqrt(3) - 1/2, b == 1/2*I*sqrt(3) + 1/2], [a == 1/2*I*sqrt(3) + 1/2, b == 1/2*I*sqrt(3) + 1/2], [a == 1/2*I*sqrt(3) - 1/2, b == -1/2*sqrt(3) - 1/2*I], [a == -1/2*I*sqrt(3) + 1/2, b == -1/2*sqrt(3) - 1/2*I], [a == 1/2*I*sqrt(3) - 1/2, b == 1/2*sqrt(3) + 1/2*I], [a == -1/2*I*sqrt(3) + 1/2, b == 1/2*sqrt(3) + 1/2*I], [a == -1/2*sqrt(3) + 1/2*I, b == -1/2*sqrt(3) + 1/2*I], [a == 1/2*sqrt(3) - 1/2*I, b == -1/2*sqrt(3) + 1/2*I], [a == -1/2*sqrt(3) + 1/2*I, b == 1/2*sqrt(3) - 1/2*I], [a == 1/2*sqrt(3) - 1/2*I, b == 1/2*sqrt(3) - 1/2*I], [a == 0, b == (-I + 1)], [a == 0, b == (I + 1)], [a == 0, b == (-I - 1)], [a == 0, b == (I - 1)], [a == (-I + 1), b == 

In [24]:
GpIII_Elts = [ [ [ -1, 0, 0 ], [ 0, -1, 0 ], [ 0, 0, 1 ] ],
  [ [ -1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, -1, 0 ], [ -1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, -1, 0 ], [ 1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, 1, 0 ], [ -1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, 1, 0 ], [ 1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, -E(4), 0 ], [ -E(4), 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, -E(4), 0 ], [ E(4), 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, E(4), 0 ], [ -E(4), 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, E(4), 0 ], [ E(4), 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 1, 0, 0 ], [ 0, -1, 0 ], [ 0, 0, 1 ] ],
  [ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ] ],
  [ [ -1/2-1/2*E(4), -1/2-1/2*E(4), 0 ], [ -1/2+1/2*E(4), 1/2-1/2*E(4), 0 ],
      [ 0, 0, E(3)^2 ] ],
  [ [ -1/2-1/2*E(4), -1/2-1/2*E(4), 0 ], [ 1/2-1/2*E(4), -1/2+1/2*E(4), 0 ],
      [ 0, 0, E(3)^2 ] ],
  [ [ -1/2-1/2*E(4), -1/2+1/2*E(4), 0 ], [ -1/2-1/2*E(4), 1/2-1/2*E(4), 0 ],
      [ 0, 0, E(3) ] ],
  [ [ -1/2-1/2*E(4), -1/2+1/2*E(4), 0 ], [ 1/2+1/2*E(4), -1/2+1/2*E(4), 0 ],
      [ 0, 0, E(3) ] ],
  [ [ -1/2-1/2*E(4), 1/2-1/2*E(4), 0 ], [ -1/2-1/2*E(4), -1/2+1/2*E(4), 0 ],
      [ 0, 0, E(3) ] ],
  [ [ -1/2-1/2*E(4), 1/2-1/2*E(4), 0 ], [ 1/2+1/2*E(4), 1/2-1/2*E(4), 0 ],
      [ 0, 0, E(3) ] ],
  [ [ -1/2-1/2*E(4), 1/2+1/2*E(4), 0 ], [ -1/2+1/2*E(4), -1/2+1/2*E(4), 0 ],
      [ 0, 0, E(3)^2 ] ],
  [ [ -1/2-1/2*E(4), 1/2+1/2*E(4), 0 ], [ 1/2-1/2*E(4), 1/2-1/2*E(4), 0 ],
      [ 0, 0, E(3)^2 ] ],
  [ [ -1/2+1/2*E(4), -1/2-1/2*E(4), 0 ], [ -1/2+1/2*E(4), 1/2+1/2*E(4), 0 ],
      [ 0, 0, E(3) ] ],
  [ [ -1/2+1/2*E(4), -1/2-1/2*E(4), 0 ], [ 1/2-1/2*E(4), -1/2-1/2*E(4), 0 ],
      [ 0, 0, E(3) ] ],
  [ [ -1/2+1/2*E(4), -1/2+1/2*E(4), 0 ], [ -1/2-1/2*E(4), 1/2+1/2*E(4), 0 ],
      [ 0, 0, E(3)^2 ] ],
  [ [ -1/2+1/2*E(4), -1/2+1/2*E(4), 0 ], [ 1/2+1/2*E(4), -1/2-1/2*E(4), 0 ],
      [ 0, 0, E(3)^2 ] ],
  [ [ -1/2+1/2*E(4), 1/2-1/2*E(4), 0 ], [ -1/2-1/2*E(4), -1/2-1/2*E(4), 0 ],
      [ 0, 0, E(3)^2 ] ],
  [ [ -1/2+1/2*E(4), 1/2-1/2*E(4), 0 ], [ 1/2+1/2*E(4), 1/2+1/2*E(4), 0 ],
      [ 0, 0, E(3)^2 ] ],
  [ [ -1/2+1/2*E(4), 1/2+1/2*E(4), 0 ], [ -1/2+1/2*E(4), -1/2-1/2*E(4), 0 ],
      [ 0, 0, E(3) ] ],
  [ [ -1/2+1/2*E(4), 1/2+1/2*E(4), 0 ], [ 1/2-1/2*E(4), 1/2+1/2*E(4), 0 ],
      [ 0, 0, E(3) ] ], [ [ -E(4), 0, 0 ], [ 0, -E(4), 0 ], [ 0, 0, 1 ] ],
  [ [ -E(4), 0, 0 ], [ 0, E(4), 0 ], [ 0, 0, 1 ] ],
  [ [ E(4), 0, 0 ], [ 0, -E(4), 0 ], [ 0, 0, 1 ] ],
  [ [ E(4), 0, 0 ], [ 0, E(4), 0 ], [ 0, 0, 1 ] ],
  [ [ 1/2-1/2*E(4), -1/2-1/2*E(4), 0 ], [ -1/2+1/2*E(4), -1/2-1/2*E(4), 0 ],
      [ 0, 0, E(3) ] ],
  [ [ 1/2-1/2*E(4), -1/2-1/2*E(4), 0 ], [ 1/2-1/2*E(4), 1/2+1/2*E(4), 0 ],
      [ 0, 0, E(3) ] ],
  [ [ 1/2-1/2*E(4), -1/2+1/2*E(4), 0 ], [ -1/2-1/2*E(4), -1/2-1/2*E(4), 0 ],
      [ 0, 0, E(3)^2 ] ],
  [ [ 1/2-1/2*E(4), -1/2+1/2*E(4), 0 ], [ 1/2+1/2*E(4), 1/2+1/2*E(4), 0 ],
      [ 0, 0, E(3)^2 ] ],
  [ [ 1/2-1/2*E(4), 1/2-1/2*E(4), 0 ], [ -1/2-1/2*E(4), 1/2+1/2*E(4), 0 ],
      [ 0, 0, E(3)^2 ] ],
  [ [ 1/2-1/2*E(4), 1/2-1/2*E(4), 0 ], [ 1/2+1/2*E(4), -1/2-1/2*E(4), 0 ],
      [ 0, 0, E(3)^2 ] ],
  [ [ 1/2-1/2*E(4), 1/2+1/2*E(4), 0 ], [ -1/2+1/2*E(4), 1/2+1/2*E(4), 0 ],
      [ 0, 0, E(3) ] ],
  [ [ 1/2-1/2*E(4), 1/2+1/2*E(4), 0 ], [ 1/2-1/2*E(4), -1/2-1/2*E(4), 0 ],
      [ 0, 0, E(3) ] ],
  [ [ 1/2+1/2*E(4), -1/2-1/2*E(4), 0 ], [ -1/2+1/2*E(4), -1/2+1/2*E(4), 0 ],
      [ 0, 0, E(3)^2 ] ],
  [ [ 1/2+1/2*E(4), -1/2-1/2*E(4), 0 ], [ 1/2-1/2*E(4), 1/2-1/2*E(4), 0 ],
      [ 0, 0, E(3)^2 ] ],
  [ [ 1/2+1/2*E(4), -1/2+1/2*E(4), 0 ], [ -1/2-1/2*E(4), -1/2+1/2*E(4), 0 ],
      [ 0, 0, E(3) ] ],
  [ [ 1/2+1/2*E(4), -1/2+1/2*E(4), 0 ], [ 1/2+1/2*E(4), 1/2-1/2*E(4), 0 ],
      [ 0, 0, E(3) ] ],
  [ [ 1/2+1/2*E(4), 1/2-1/2*E(4), 0 ], [ -1/2-1/2*E(4), 1/2-1/2*E(4), 0 ],
      [ 0, 0, E(3) ] ],
  [ [ 1/2+1/2*E(4), 1/2-1/2*E(4), 0 ], [ 1/2+1/2*E(4), -1/2+1/2*E(4), 0 ],
      [ 0, 0, E(3) ] ],
  [ [ 1/2+1/2*E(4), 1/2+1/2*E(4), 0 ], [ -1/2+1/2*E(4), 1/2-1/2*E(4), 0 ],
      [ 0, 0, E(3)^2 ] ],
  [ [ 1/2+1/2*E(4), 1/2+1/2*E(4), 0 ], [ 1/2-1/2*E(4), -1/2+1/2*E(4), 0 ],
      [ 0, 0, E(3)^2 ] ] ]

keys = ['r' + str(j) for j in range(0,48)]
GpIII_Dict = {}
for j in range(0,48):
    GpIII_Dict.update({keys[j]:matrix(CC,GpIII_Elts[j],immutable=True)});

GpIII_Dict;

In [25]:
QuarticIIIBis = [x + (-0.50000084 + 0.50000314*I)*z,
x + (-0.50000314 - 0.50000084*I)*z,
x - y + (0.500000000000000 - 0.866030000000000*I)*z,
x - y + (-0.500000000000000 + 0.866030000000000*I)*z,
x - y + (-0.866030000000000 - 0.500000000000000*I)*z,
x - y + (0.866030000000000 + 0.500000000000000*I)*z,
x + (0.50000314 + 0.50000084*I)*z,
x + (0.50000084 - 0.50000314*I)*z,
y + (0.50000314 + 0.50000084*I)*z,
y + (-0.50000084 + 0.50000314*I)*z,
y + (0.50000084 - 0.50000314*I)*z,
y + (-0.50000314 - 0.50000084*I)*z,
x + 1.00000000000000*I*y + (0.500000000000000 + 0.866030000000000*I)*z,
x + 1.00000000000000*I*y + (0.866030000000000 - 0.500000000000000*I)*z,
x + 1.00000000000000*I*y + (-0.866030000000000 + 0.500000000000000*I)*z,
x + 1.00000000000000*I*y + (-0.500000000000000 - 0.866030000000000*I)*z,
x + (-1.00000000000000*I)*y + (0.500000000000000 + 0.866030000000000*I)*z,
x + (-1.00000000000000*I)*y + (0.866030000000000 - 0.500000000000000*I)*z,
x + (-1.00000000000000*I)*y + (-0.866030000000000 + 0.500000000000000*I)*z,
x + (-1.00000000000000*I)*y + (-0.500000000000000 - 0.866030000000000*I)*z,
x + y + (0.500000000000000 - 0.866030000000000*I)*z,
x + y + (-0.866030000000000 - 0.500000000000000*I)*z,
x + y + (-0.500000000000000 + 0.866030000000000*I)*z,
x + y + (0.866030000000000 + 0.500000000000000*I)*z,
x + (-0.36602540 - 0.36602540*I)*y,
x + (0.36602540 + 0.36602540*I)*y,
x + (-1.3660254 + 1.3660254*I)*y,
x + (1.3660254 - 1.3660254*I)*y]


In [26]:
investigate_orbit(QuarticIIIBis[27],GpIII_Dict)

x + (1.3660254 - 1.3660254*I)*y
x + (1.36602540000000 - 1.36602540000000*I)*y
x + (0.36602541 + 0.36602541*I)*y
x + (-0.36602541 - 0.36602541*I)*y
x + (0.36602540 + 0.36602540*I)*y
x + (-0.36602540 - 0.36602540*I)*y
x + (1.3660254 - 1.3660254*I)*y
x + (-0.36602540 - 0.36602540*I)*y
x + (0.36602540 + 0.36602540*I)*y
x + (1.3660254 - 1.3660254*I)*y
x + (-1.3660254 + 1.3660254*I)*y
x + (-1.3660254 + 1.3660254*I)*y
x + (-1.3660254 + 1.3660254*I)*y
I think isotropy is generated by:

r0
r11
r28
r31


In [27]:
GpIII_Dict['r0']

[-1.00000000000000 0.000000000000000 0.000000000000000]
[0.000000000000000 -1.00000000000000 0.000000000000000]
[0.000000000000000 0.000000000000000  1.00000000000000]

# Type IV: The $S_4$ quartic<a name="S4"></a>
We have an $S_4$-symmetric quartic with equation
$$X^4 + Y^4 + Z^4 + a(X^2 Y^2 + Y^2 Z^2 + Z^2 X^2)$$
and symmetries ...

## (IV): Old $S_4$

In [28]:
F=QQ; T.<x,y,z>=PolynomialRing(F)
S4_Quartic = x^4 + y^4 + z^4 + (x^2*y^2 + y^2*z^2 + z^2*x^2)
S4_Bitangents = compute_bitangents(S4_Quartic); S4_Bitangents
S4_Bitangents = [normalize(b) for b in S4_Bitangents]

# gena = matrix(CC,[[0,0,1],[1,0,0],[0,1,0]],immutable=True)
# genb = matrix(CC,[[0,-1,0],[1,0,0],[0,0,1]],immutable=True)
gena = [[0,0,1],[1,0,0],[0,1,0]]
genb = [[0,-1,0],[1,0,0],[0,0,1]]

S4_elts = [ [ [ -1, 0, 0 ], [ 0, -1, 0 ], [ 0, 0, 1 ] ], [ [ -1, 0, 0 ], [ 0, 0, -1 ], [ 0, -1, 0 ] ], [ [ -1, 0, 0 ], [ 0, 0, 1 ], [ 0, 1, 0 ] ],
  [ [ -1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, -1 ] ], [ [ 0, -1, 0 ], [ -1, 0, 0 ], [ 0, 0, -1 ] ], [ [ 0, -1, 0 ], [ 0, 0, -1 ], [ 1, 0, 0 ] ],
  [ [ 0, -1, 0 ], [ 0, 0, 1 ], [ -1, 0, 0 ] ], [ [ 0, -1, 0 ], [ 1, 0, 0 ], [ 0, 0, 1 ] ], [ [ 0, 0, -1 ], [ -1, 0, 0 ], [ 0, 1, 0 ] ],
  [ [ 0, 0, -1 ], [ 0, -1, 0 ], [ -1, 0, 0 ] ], [ [ 0, 0, -1 ], [ 0, 1, 0 ], [ 1, 0, 0 ] ], [ [ 0, 0, -1 ], [ 1, 0, 0 ], [ 0, -1, 0 ] ],
  [ [ 0, 0, 1 ], [ -1, 0, 0 ], [ 0, -1, 0 ] ], [ [ 0, 0, 1 ], [ 0, -1, 0 ], [ 1, 0, 0 ] ], [ [ 0, 0, 1 ], [ 0, 1, 0 ], [ -1, 0, 0 ] ],
  [ [ 0, 0, 1 ], [ 1, 0, 0 ], [ 0, 1, 0 ] ], [ [ 0, 1, 0 ], [ -1, 0, 0 ], [ 0, 0, 1 ] ], [ [ 0, 1, 0 ], [ 0, 0, -1 ], [ -1, 0, 0 ] ],
  [ [ 0, 1, 0 ], [ 0, 0, 1 ], [ 1, 0, 0 ] ], [ [ 0, 1, 0 ], [ 1, 0, 0 ], [ 0, 0, -1 ] ], [ [ 1, 0, 0 ], [ 0, -1, 0 ], [ 0, 0, -1 ] ],
  [ [ 1, 0, 0 ], [ 0, 0, -1 ], [ 0, 1, 0 ] ], [ [ 1, 0, 0 ], [ 0, 0, 1 ], [ 0, -1, 0 ] ], [ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ] ] ];

keys = ['r' + str(j) for j in range(0,24)]
S4_dict = {}
for j in range(0,24):
    S4_dict.update({keys[j]:matrix(CC,S4_elts[j],immutable=True)});

S4_Bitangents

The quartic has 28 bitangets, stored in 'Bitangents', and 4 real bitangents, stored in 'RealBitangents'.


[y + (-0.57735027 - 0.81649658*I)*z,
 y + (-0.57735027 + 0.81649658*I)*z,
 y + (0.57735027 + 0.81649658*I)*z,
 y + (0.57735027 - 0.81649658*I)*z,
 x + (-1.41421*I)*y - z,
 x + 1.41421*I*y + z,
 x + 1.41421*I*y - z,
 x + (-1.41421*I)*y + z,
 x + y + 1.41421*I*z,
 x - y + (-1.41421*I)*z,
 x - y + 1.41421*I*z,
 x + y + (-1.41421*I)*z,
 x + y - z,
 x - y + z,
 x + 0.70711*I*y + (-0.70711*I)*z,
 x + (-0.70711*I)*y + 0.70711*I*z,
 x + (-0.57735027 - 0.81649658*I)*z,
 x + (-0.57735027 + 0.81649658*I)*z,
 x + (0.57735027 + 0.81649658*I)*z,
 x + (0.57735027 - 0.81649658*I)*z,
 x - y - z,
 x + y + z,
 x + (-0.70711*I)*y + (-0.70711*I)*z,
 x + 0.70711*I*y + 0.70711*I*z,
 x + (-0.57735027 - 0.81649658*I)*y,
 x + (-0.57735027 + 0.81649658*I)*y,
 x + (0.57735027 + 0.81649658*I)*y,
 x + (0.57735027 - 0.81649658*I)*y]

In [29]:
e_isotropy = []
C2_even = [[24,25,26,27,16,17,18,19,0,1,2,3]]
C2_odd = [[4,5,6,7,14,15,22,23,8,9,10,11]]
S3 = [[12,13,20,21]]
orbit_indices = e_isotropy + C2_even + C2_odd + S3
all_orbit_indices = flatten(orbit_indices)
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(S4_Bitangents[b_index])
    list_of_bitangent_orbits.append(orbit)

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(S4_Bitangents)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(S4_Bitangents[j]))

Orbits of bitangents: [[x + (-0.57735027 - 0.81649658*I)*y, x + (-0.57735027 + 0.81649658*I)*y, x + (0.57735027 + 0.81649658*I)*y, x + (0.57735027 - 0.81649658*I)*y, x + (-0.57735027 - 0.81649658*I)*z, x + (-0.57735027 + 0.81649658*I)*z, x + (0.57735027 + 0.81649658*I)*z, x + (0.57735027 - 0.81649658*I)*z, y + (-0.57735027 - 0.81649658*I)*z, y + (-0.57735027 + 0.81649658*I)*z, y + (0.57735027 + 0.81649658*I)*z, y + (0.57735027 - 0.81649658*I)*z], [x + (-1.41421*I)*y - z, x + 1.41421*I*y + z, x + 1.41421*I*y - z, x + (-1.41421*I)*y + z, x + 0.70711*I*y + (-0.70711*I)*z, x + (-0.70711*I)*y + 0.70711*I*z, x + (-0.70711*I)*y + (-0.70711*I)*z, x + 0.70711*I*y + 0.70711*I*z, x + y + 1.41421*I*z, x - y + (-1.41421*I)*z, x - y + 1.41421*I*z, x + y + (-1.41421*I)*z], [x + y - z, x - y + z, x - y - z, x + y + z]]

Remaining bitangents:



In [30]:
investigate_orbit_S4(S4_Bitangents[12],S4_dict)

x + y + z
x - y + z
x - y - z
x + y - z
I think isotropy is generated by:

[[-1, 0, 0], [0, 0, 1], [0, 1, 0]],
[[0, -1, 0], [-1, 0, 0], [0, 0, -1]],
[[0, 0, -1], [1, 0, 0], [0, -1, 0]],
[[0, 0, 1], [0, -1, 0], [1, 0, 0]],
[[0, 1, 0], [0, 0, -1], [-1, 0, 0]],
[[1, 0, 0], [0, 1, 0], [0, 0, 1]],


## (IV): New $S_4$, real points + bitangents

In [31]:
F=QQ; T.<x,y,z>=PolynomialRing(F)
S4_Quartic_new = x^4 + y^4 + z^4 -3*(x^2*y^2 + y^2*z^2 + z^2*x^2)
S4_Bitangents_new = compute_bitangents(S4_Quartic_new);S4_Bitangents_new
S4_Bitangents_new = [normalize(b) for b in S4_Bitangents_new];S4_Bitangents_new

The quartic has 28 bitangets, stored in 'Bitangents', and 16 real bitangents, stored in 'RealBitangents'.


[y + (-0.41421356*I)*z,
 y + 0.41421356*I*z,
 y + 2.4142136*I*z,
 y + (-2.4142136*I)*z,
 x + 1.41421*y - z,
 x + (-1.41421)*y + z,
 x + y + (-1.41421)*z,
 x - y + 1.41421*z,
 x - y + (-1.41421)*z,
 x + y + 1.41421*z,
 x + (-1.41421)*y - z,
 x + 1.41421*y + z,
 x + 0.70711*y + (-0.70711)*z,
 x + y - z,
 x - y + z,
 x + (-0.70711)*y + 0.70711*z,
 x + (-0.41421356*I)*z,
 x + 0.41421356*I*z,
 x + 2.4142136*I*z,
 x + (-2.4142136*I)*z,
 x + (-0.70711)*y + (-0.70711)*z,
 x - y - z,
 x + y + z,
 x + 0.70711*y + 0.70711*z,
 x + (-0.41421356*I)*y,
 x + 0.41421356*I*y,
 x + 2.4142136*I*y,
 x + (-2.4142136*I)*y]

## $S_4$ dictionary

In [32]:
S4_elts = [ [ [ -1, 0, 0 ], [ 0, -1, 0 ], [ 0, 0, 1 ] ], [ [ -1, 0, 0 ], [ 0, 0, -1 ], [ 0, -1, 0 ] ], [ [ -1, 0, 0 ], [ 0, 0, 1 ], [ 0, 1, 0 ] ],
  [ [ -1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, -1 ] ], [ [ 0, -1, 0 ], [ -1, 0, 0 ], [ 0, 0, -1 ] ], [ [ 0, -1, 0 ], [ 0, 0, -1 ], [ 1, 0, 0 ] ],
  [ [ 0, -1, 0 ], [ 0, 0, 1 ], [ -1, 0, 0 ] ], [ [ 0, -1, 0 ], [ 1, 0, 0 ], [ 0, 0, 1 ] ], [ [ 0, 0, -1 ], [ -1, 0, 0 ], [ 0, 1, 0 ] ],
  [ [ 0, 0, -1 ], [ 0, -1, 0 ], [ -1, 0, 0 ] ], [ [ 0, 0, -1 ], [ 0, 1, 0 ], [ 1, 0, 0 ] ], [ [ 0, 0, -1 ], [ 1, 0, 0 ], [ 0, -1, 0 ] ],
  [ [ 0, 0, 1 ], [ -1, 0, 0 ], [ 0, -1, 0 ] ], [ [ 0, 0, 1 ], [ 0, -1, 0 ], [ 1, 0, 0 ] ], [ [ 0, 0, 1 ], [ 0, 1, 0 ], [ -1, 0, 0 ] ],
  [ [ 0, 0, 1 ], [ 1, 0, 0 ], [ 0, 1, 0 ] ], [ [ 0, 1, 0 ], [ -1, 0, 0 ], [ 0, 0, 1 ] ], [ [ 0, 1, 0 ], [ 0, 0, -1 ], [ -1, 0, 0 ] ],
  [ [ 0, 1, 0 ], [ 0, 0, 1 ], [ 1, 0, 0 ] ], [ [ 0, 1, 0 ], [ 1, 0, 0 ], [ 0, 0, -1 ] ], [ [ 1, 0, 0 ], [ 0, -1, 0 ], [ 0, 0, -1 ] ],
  [ [ 1, 0, 0 ], [ 0, 0, -1 ], [ 0, 1, 0 ] ], [ [ 1, 0, 0 ], [ 0, 0, 1 ], [ 0, -1, 0 ] ], [ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ] ] ];

keys = ['r' + str(j) for j in range(0,24)]
S4_dict = {}
for j in range(0,24):
    S4_dict.update({keys[j]:matrix(CC,S4_elts[j],immutable=True)});

## IV.3 Orbits

In [33]:
e_isotropy = []
C2_even = [[0,1,2,3,18,19,26,27,25,24,16,17]]
C2_odd = [[10,4,5,6,11,9,12,20,23,15,8,7]]
S3 = [[13,14,21,22]]
orbit_indices = e_isotropy + C2_even + C2_odd + S3
all_orbit_indices = flatten(orbit_indices)
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(S4_Bitangents_new[b_index])
    list_of_bitangent_orbits.append(orbit)

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(S4_Bitangents_new)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(S4_Bitangents_new[j]))

Orbits of bitangents: [[y + (-0.41421356*I)*z, y + 0.41421356*I*z, y + 2.4142136*I*z, y + (-2.4142136*I)*z, x + 2.4142136*I*z, x + (-2.4142136*I)*z, x + 2.4142136*I*y, x + (-2.4142136*I)*y, x + 0.41421356*I*y, x + (-0.41421356*I)*y, x + (-0.41421356*I)*z, x + 0.41421356*I*z], [x + (-1.41421)*y - z, x + 1.41421*y - z, x + (-1.41421)*y + z, x + y + (-1.41421)*z, x + 1.41421*y + z, x + y + 1.41421*z, x + 0.70711*y + (-0.70711)*z, x + (-0.70711)*y + (-0.70711)*z, x + 0.70711*y + 0.70711*z, x + (-0.70711)*y + 0.70711*z, x - y + (-1.41421)*z, x - y + 1.41421*z], [x + y - z, x - y + z, x - y - z, x + y + z]]

Remaining bitangents:



In [34]:
investigate_orbit_S4(S4_Bitangents_new[13],S4_dict)

x + y + z
x - y + z
x - y - z
x + y - z
I think isotropy is generated by:

[[-1, 0, 0], [0, 0, 1], [0, 1, 0]],
[[0, -1, 0], [-1, 0, 0], [0, 0, -1]],
[[0, 0, -1], [1, 0, 0], [0, -1, 0]],
[[0, 0, 1], [0, -1, 0], [1, 0, 0]],
[[0, 1, 0], [0, 0, -1], [-1, 0, 0]],
[[1, 0, 0], [0, 1, 0], [0, 0, 1]],


# Type V: The $(16,13)$ quartic<a name="1613"></a>
We have an $G$-symmetric quartic with equation
$$X^4 + Y^4 + Z^4 + aX^2 Y^2$$
This is Type V in Dolgachev. and symmetries ...

## (V) Old

In [35]:
V_Quartic = x^4 + y^4 + z^4 + x^2*y^2
V_Bitangents = compute_bitangents(V_Quartic)
V_Bitangents = [normalize(b) for b in V_Bitangents]

#############

gen1 = matrix(CC,[[-1,0,0],[0,1,0],[0,0,1]])
gen2 = matrix(CC,[[I,0,0],[0,-I,0],[0,0,1]])
gen3 = matrix(CC,[[0,-1,0],[1,0,0],[0,0,1]])

VGrpElts = [  [ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ] ],
    [ [ -1, 0, 0 ], [ 0, -1, 0 ], [ 0, 0, 1 ] ],
  [ [ -1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, -1, 0 ], [ -1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, -1, 0 ], [ 1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, 1, 0 ], [ -1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, 1, 0 ], [ 1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, -I, 0 ], [ -I, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, -I, 0 ], [ I, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, I, 0 ], [ -I, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, I, 0 ], [ I, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 1, 0, 0 ], [ 0, -1, 0 ], [ 0, 0, 1 ] ],
  [ [ -I, 0, 0 ], [ 0, -I, 0 ], [ 0, 0, 1 ] ],
  [ [ -I, 0, 0 ], [ 0, I, 0 ], [ 0, 0, 1 ] ],
  [ [ I, 0, 0 ], [ 0, -I, 0 ], [ 0, 0, 1 ] ],
  [ [ I, 0, 0 ], [ 0, I, 0 ], [ 0, 0, 1 ] ] ]

keys = ['r' + str(j) for j in range(0,16)]
VGrp_dict = {}
for j in range(0,16):
    VGrp_dict.update({keys[j]:matrix(CC,VGrpElts[j],immutable=True)});



The quartic has 28 bitangets, stored in 'Bitangents', and 4 real bitangents, stored in 'RealBitangents'.


In [36]:
###########
someC2 = [[0,1,2,3,20,21,22,23]]
# Stabilized by VGrp_dict['r2']
#      matrix_to_list(VGrp_dict['r2'])

# this is not conjugate to the first,
# stabilized by VGrp_dict['r9']
anotherC2 = [[4,11,12,15,5,13,10,14]]

# stabilized by VGrp_dict['r6'], also not conjugate to the first
yetanotherC2 = [[18,6,16,8,9,17,7,19]]

# the last isotropy is the center of the group, isomorphic to C4
center_C4 = [[24,25,26,27]]

orbit_indices = someC2 + anotherC2 + yetanotherC2 + center_C4

all_orbit_indices = flatten(orbit_indices)
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(V_Bitangents[b_index])
    list_of_bitangent_orbits.append(orbit)

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(V_Bitangents)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(V_Bitangents[j]))

Orbits of bitangents: [[1.0000000*y + (-0.75983568 - 0.75983568*I)*z, 1.0000000*y + (-0.75983568 + 0.75983568*I)*z, 1.0000000*y + (0.75983568 + 0.75983568*I)*z, 1.0000000*y + (0.75983568 - 0.75983568*I)*z, 1.0000000*x + (-0.75983568 - 0.75983568*I)*z, 1.0000000*x + (-0.75983568 + 0.75983568*I)*z, 1.0000000*x + (0.75983568 + 0.75983568*I)*z, 1.0000000*x + (0.75983568 - 0.75983568*I)*z], [x + (-1.0*I)*y + 0.75984*I*z, x + (-1.0*I)*y + (-0.75984*I)*z, x + (-1.0*I)*y + (-0.75984)*z, x + (-1.0*I)*y + 0.75984*z, x + 1.0*I*y + (-0.75984*I)*z, x + 1.0*I*y + 0.75984*z, x + 1.0*I*y + 0.75984*I*z, x + 1.0*I*y + (-0.75984)*z], [x + y + 1.31607*I*z, x + y + (-1.31607)*z, x - y + 1.31607*I*z, x - y + (-1.31607)*z, x + y + 1.31607*z, x + y + (-1.31607*I)*z, x - y + 1.31607*z, x - y + (-1.31607*I)*z], [x + (-0.50000000 - 0.86602540*I)*y, x + (-0.50000000 + 0.86602540*I)*y, x + (0.50000000 + 0.86602540*I)*y, x + (0.50000000 - 0.86602540*I)*y]]

Remaining bitangents:



## (V) New curve and bitangents:

In [37]:
R.<x,y> = PolynomialRing(RR)
maxval=5
V_Quartic_affine = x^4 + y^4 + 1 - 4*x^2*y^2
implicit_plot(V_Quartic_affine,(-maxval,maxval),(-maxval,maxval),color='black')

Graphics object consisting of 1 graphics primitive

In [38]:
F=QQ; T.<x,y,z>=PolynomialRing(F)
V_Quartic_new = x^4 + y^4 + z^4 - 4*x^2*y^2
# V_Quartic_affine = x^4 + y^4 + 1 - 4*x^2*y^2
V_Bitangents_new = compute_bitangents(V_Quartic_new)
V_Bitangents_new = [normalize(b) for b in V_Bitangents_new]

The quartic has 28 bitangets, stored in 'Bitangents', and 8 real bitangents, stored in 'RealBitangents'.


## (V) V_group_dict

In [39]:
VGrpElts = [  [ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ] ],
    [ [ -1, 0, 0 ], [ 0, -1, 0 ], [ 0, 0, 1 ] ],
  [ [ -1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, -1, 0 ], [ -1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, -1, 0 ], [ 1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, 1, 0 ], [ -1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, 1, 0 ], [ 1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, -I, 0 ], [ -I, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, -I, 0 ], [ I, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, I, 0 ], [ -I, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, I, 0 ], [ I, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 1, 0, 0 ], [ 0, -1, 0 ], [ 0, 0, 1 ] ],
  [ [ -I, 0, 0 ], [ 0, -I, 0 ], [ 0, 0, 1 ] ],
  [ [ -I, 0, 0 ], [ 0, I, 0 ], [ 0, 0, 1 ] ],
  [ [ I, 0, 0 ], [ 0, -I, 0 ], [ 0, 0, 1 ] ],
  [ [ I, 0, 0 ], [ 0, I, 0 ], [ 0, 0, 1 ] ] ]

keys = ['r' + str(j) for j in range(0,16)]
VGrp_dict = {}
for j in range(0,16):
    VGrp_dict.update({keys[j]:matrix(CC,VGrpElts[j],immutable=True)});

## (V) Orbits of bitangents

In [40]:
###########
someC2 = [[0,1,2,3,20,21,22,23]]

anotherC2 = [[4,6,17,19,5,7,16,18]]

yetanotherC2 = []

# the last isotropy is the center of the group, isomorphic to C4
center_C4 = [[24,25,26,27]]

orbit_indices = someC2 + anotherC2 + yetanotherC2 + center_C4

all_orbit_indices = flatten(orbit_indices)
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(V_Bitangents_new[b_index])
    list_of_bitangent_orbits.append(orbit)

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(V_Bitangents_new)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(V_Bitangents_new[j]))

Orbits of bitangents: [[1.0000000*y + (-0.75983568)*z, 1.0000000*y + 0.75983568*z, 1.0000000*y + (-0.75983568*I)*z, 1.0000000*y + 0.75983568*I*z, 1.0000000*x + (-0.75983568)*z, 1.0000000*x + 0.75983568*z, 1.0000000*x + (-0.75983568*I)*z, 1.0000000*x + 0.75983568*I*z], [x + y + (-0.53728 - 0.53728*I)*z, x + y + (-0.53728 + 0.53728*I)*z, x + y + (0.53728 + 0.53728*I)*z, x + y + (0.53728 - 0.53728*I)*z, x - y + (0.53728 + 0.53728*I)*z, x - y + (0.53728 - 0.53728*I)*z, x - y + (-0.53728 - 0.53728*I)*z, x - y + (-0.53728 + 0.53728*I)*z], [x + (-0.51763809)*y, x + (-1.9318517)*y, x + 1.9318517*y, x + 0.51763809*y]]

Remaining bitangents:

8: x + (-1.0*I)*y + (-0.9306 + 0.9306*I)*z
9: x + 1.0*I*y + (0.9306 - 0.9306*I)*z
10: x + 1.0*I*y + (-0.9306 - 0.9306*I)*z
11: x + (-1.0*I)*y + (0.9306 + 0.9306*I)*z
12: x + (-1.0*I)*y + (-0.9306 - 0.9306*I)*z
13: x + 1.0*I*y + (0.9306 + 0.9306*I)*z
14: x + 1.0*I*y + (-0.9306 + 0.9306*I)*z
15: x + (-1.0*I)*y + (0.9306 - 0.9306*I)*z


In [41]:
investigate_orbit(V_Bitangents_new[0],VGrp_dict)

1.0000000*y + (-0.75983568)*z
y + 0.75983569*z
y + (-0.75983569)*z
y + (-0.75983569*I)*z
y + 0.75983569*I*z
x + 0.75983569*z
x + (-0.75983569*I)*z
x + 0.75983569*I*z
x + (-0.75983569)*z
I think isotropy is generated by:



TODO --- find the isotropy for this orbit

# Type VI: The $C_9$ quartic<a name="C9"></a>
We have an $C_9$-symmetric quartic with equation
$$X^4 + XY^3 + YZ^3$$
and symmetries ...

In [42]:
C9_Quartic = x^4 + x*y^3 + y*z^3
C9_Bitangents = compute_bitangents(C9_Quartic)
C9_Bitangents = [normalize(bitangent) for bitangent in C9_Bitangents];C9_Bitangents

rho = exp(2*pi*i/3)
omega = exp(2*pi*i/9)

C9_dict = {
    'r' : matrix(CC,[[rho,0,0],[0,1,0],[0,0,omega]],immutable=True),
    'r2' : matrix(CC,[[rho^2,0,0],[0,1,0],[0,0,omega^2]],immutable=True),
    'r3' : matrix(CC,[[1,0,0],[0,1,0],[0,0,rho]],immutable=True),
    'r4' : matrix(CC,[[rho^4,0,0],[0,1,0],[0,0,omega^4]],immutable=True),
    'r5' : matrix(CC,[[rho^5,0,0],[0,1,0],[0,0,omega^5]],immutable=True),
    'r6' : matrix(CC,[[rho^6,0,0],[0,1,0],[0,0,omega^6]],immutable=True),
    'r7' : matrix(CC,[[rho^7,0,0],[0,1,0],[0,0,omega^7]],immutable=True),
    'r8' : matrix(CC,[[rho^8,0,0],[0,1,0],[0,0,omega^8]],immutable=True),
}

The quartic has 28 bitangets, stored in 'Bitangents', and 4 real bitangents, stored in 'RealBitangents'.


In [43]:
e_isotropy = [[4,0,23,24,25,3,26,8,7],[6,1,5,20,12,21,22,19,11],[2,9,10,13,14,15,16,17,18]]
C3_isotropy = []
C9_isotropy = [[27]]
orbit_indices = e_isotropy + C3_isotropy + C9_isotropy
all_orbit_indices = flatten(orbit_indices)
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(C9_Bitangents[b_index])
    list_of_bitangent_orbits.append(orbit)

# remaining_bitangents = rescaled_C3_Bitangents

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(C9_Bitangents)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(C9_Bitangents[j]))

Orbits of bitangents: [[x + (-0.59197 + 1.02532*I)*y + (-0.07781 - 0.44126*I)*z, x + 1.18394*y + (-0.44807)*z, x + 1.18394*y + (0.22404 - 0.38804*I)*z, x + 1.18394*y + (0.22404 + 0.38804*I)*z, x + (-0.59197 - 1.02532*I)*y + (-0.34324 - 0.28802*I)*z, x + (-0.59197 - 1.02532*I)*y + (-0.07781 + 0.44126*I)*z, x + (-0.59197 + 1.02532*I)*y + (-0.34324 + 0.28802*I)*z, x + (-0.59197 - 1.02532*I)*y + (0.42105 - 0.15325*I)*z, x + (-0.59197 + 1.02532*I)*y + (0.42105 + 0.15325*I)*z], [x + (0.48258 - 0.83585*I)*y + (0.22404 + 1.27057*I)*z, x + (-0.96516)*y + 1.29017*z, x + (0.48258 + 0.83585*I)*y + (0.22404 - 1.27057*I)*z, x + (-0.96516)*y + (-0.64509 - 1.11732*I)*z, x + (0.48258 + 0.83585*I)*y + (-1.21236 + 0.44126*I)*z, x + (0.48258 + 0.83585*I)*y + (0.98833 + 0.82931*I)*z, x + (0.48258 - 0.83585*I)*y + (0.98833 - 0.82931*I)*z, x + (-0.96516)*y + (-0.64509 + 1.11732*I)*z, x + (0.48258 - 0.83585*I)*y + (-1.21236 - 0.44126*I)*z], [x + (-0.21878)*y + 0.68649*z, x + (0.10939 + 0.18947*I)*y + (0.11921

In [44]:
investigate_orbit(C9_Bitangents[27],C9_dict)

y
I think isotropy is generated by:

r
r2
r3
r4
r5
r6
r7
r8


# Type VII: The $D_4$ quartic<a name="D4"></a>
We have an $D_4$-symmetric quartic with equation
$$X^4 + Y^4 + Z^4 + aX^2 Y^2 + bXYZ^2$$
and symmetries ...

## (VII) Old:

In [45]:
D4_Quartic = x^4 + y^4 + z^4 + x^2*y^2 + x*y*z^2
D4_Bitangents = compute_bitangents(D4_Quartic)
D4_Bitangents = [normalize(bitangent) for bitangent in D4_Bitangents]

The quartic has 28 bitangets, stored in 'Bitangents', and 4 real bitangents, stored in 'RealBitangents'.


In [46]:
D8_dict = {
    'r' : matrix(CC,[[I,0,0],[0,-I,0],[0,0,1]],immutable=True),
    'r2' : matrix(CC,[[-1,0,0],[0,-1,0],[0,0,1]],immutable=True),
    'r3' : matrix(CC,[[-I,0,0],[0,I,0],[0,0,1]],immutable=True),
    's' : matrix(CC,[[0,1,0],[1,0,0],[0,0,1]],immutable=True),
    'rs' : matrix(CC,[[0,I,0],[-I,0,0],[0,0,1]],immutable=True),
    'r2s' : matrix(CC,[[0,-1,0],[-1,0,0],[0,0,1]],immutable=True),
    'r3s' : matrix(CC,[[0,-I,0],[I,0,0],[0,0,1]],immutable=True)
}


In [47]:
# Ordered by isotropy, these are all transitive D8-sets
trivial = [[16,17,11,20,10,6,21,7]]
centerC2 = [[24,25,26,27]]
otherC2 = [[0,12,3,13],[1,2,14,15],[4,9,18,23],[5,8,19,22]]
K4 = []
C4 = []
D8 = []

orbit_indices = trivial + centerC2 + otherC2 + K4 + C4 + D8
all_orbit_indices = flatten(orbit_indices)
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(D4_Bitangents[b_index])
    list_of_bitangent_orbits.append(orbit)

# remaining_bitangents = rescaled_C3_Bitangents

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(D4_Bitangents)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(D4_Bitangents[j]))

Orbits of bitangents: [[x + (-0.27818*I)*y + (-0.74589 - 0.74589*I)*z, x + 0.27818*I*y + (-0.74589 + 0.74589*I)*z, x + (-0.27818*I)*y + (0.74589 + 0.74589*I)*z, x + (-3.5948*I)*y + (-2.68134 - 2.68134*I)*z, x + 0.27818*I*y + (0.74589 - 0.74589*I)*z, x + 3.5948*I*y + (2.68134 - 2.68134*I)*z, x + 3.5948*I*y + (-2.68134 + 2.68134*I)*z, x + (-3.5948*I)*y + (2.68134 + 2.68134*I)*z], [x + (-0.55901699 - 0.82915620*I)*y, x + (-0.55901699 + 0.82915620*I)*y, x + (0.55901699 + 0.82915620*I)*y, x + (0.55901699 - 0.82915620*I)*y], [x - y + 0.96772*z, x + y + (-0.96772*I)*z, x - y + (-0.96772)*z, x + y + 0.96772*I*z], [x + y + (-1.71362)*z, x + y + 1.71362*z, x - y + 1.71362*I*z, x - y + (-1.71362*I)*z], [x + 1.0*I*y + (0.21532 - 0.77404*I)*z, x + (-1.0*I)*y + (0.77404 + 0.21532*I)*z, x + (-1.0*I)*y + (-0.77404 - 0.21532*I)*z, x + 1.0*I*y + (-0.21532 + 0.77404*I)*z], [x + (-1.0*I)*y + (0.21532 + 0.77404*I)*z, x + 1.0*I*y + (0.77404 - 0.21532*I)*z, x + 1.0*I*y + (-0.77404 + 0.21532*I)*z, x + (-1.0*I

So we get, where $C_2^Z \le D_8$ denotes the center subgroup, and $C_2$ without a decoration denotes the non-center conjugacy subgroup which is cyclic of order two:
$$[D_8/e] + 4[D_8/C_2] + [D_8/C_2^Z]$$

## (VII) New equation

In [48]:
R.<x,y> = PolynomialRing(RR)
maxval=5
D4_Quartic_affine = x^4 + y^4 + 1 - 3*x^2*y^2 + x*y
implicit_plot(D4_Quartic_affine,(-maxval,maxval),(-maxval,maxval),color='black')

Graphics object consisting of 1 graphics primitive

In [49]:
F=QQ; T.<x,y,z>=PolynomialRing(F)
D4_Quartic_new = x^4 + y^4 + z^4 - 3*x^2*y^2 + x*y*z^2
D4_Bitangents_new = compute_bitangents(D4_Quartic_new)
D4_Bitangents_new = [normalize(bitangent) for bitangent in D4_Bitangents_new];D4_Bitangents_new

The quartic has 28 bitangets, stored in 'Bitangents', and 8 real bitangents, stored in 'RealBitangents'.


[x + (-4.79129)*y + 4.3778*z,
 x + (-0.20871)*y + 0.9137*z,
 x + (-0.20871)*y + (-0.9137)*z,
 x + (-4.79129)*y + (-4.3778)*z,
 x + y + (-0.59161 - 0.3873*I)*z,
 x + y + (-0.59161 + 0.3873*I)*z,
 x - y + (0.3873 + 0.59161*I)*z,
 x - y + (0.3873 - 0.59161*I)*z,
 x + 1.0*I*y + (0.86603 - 0.86603*I)*z,
 x + (-1.0*I)*y + (0.86603 + 0.86603*I)*z,
 x + (-1.0*I)*y + (-1.32288 + 1.32288*I)*z,
 x + 1.0*I*y + (-1.32288 - 1.32288*I)*z,
 x + 0.20871*y + (-0.9137*I)*z,
 x + 0.20871*y + 0.9137*I*z,
 x + 4.79129*y + (-4.3778*I)*z,
 x + 4.79129*y + 4.3778*I*z,
 x + 1.0*I*y + (1.32288 + 1.32288*I)*z,
 x + (-1.0*I)*y + (1.32288 - 1.32288*I)*z,
 x + (-1.0*I)*y + (-0.86603 - 0.86603*I)*z,
 x + 1.0*I*y + (-0.86603 + 0.86603*I)*z,
 x - y + (-0.3873 + 0.59161*I)*z,
 x - y + (-0.3873 - 0.59161*I)*z,
 x + y + (0.59161 - 0.3873*I)*z,
 x + y + (0.59161 + 0.3873*I)*z,
 x + (-0.58662693)*y,
 x + (-1.7046609)*y,
 x + 1.7046609*y,
 x + 0.58662693*y]

## (VII) Dict

In [50]:
D8_dict = {
    'r' : matrix(CC,[[I,0,0],[0,-I,0],[0,0,1]],immutable=True),
    'r2' : matrix(CC,[[-1,0,0],[0,-1,0],[0,0,1]],immutable=True),
    'r3' : matrix(CC,[[-I,0,0],[0,I,0],[0,0,1]],immutable=True),
    's' : matrix(CC,[[0,1,0],[1,0,0],[0,0,1]],immutable=True),
    'rs' : matrix(CC,[[0,I,0],[-I,0,0],[0,0,1]],immutable=True),
    'r2s' : matrix(CC,[[0,-1,0],[-1,0,0],[0,0,1]],immutable=True),
    'r3s' : matrix(CC,[[0,-I,0],[I,0,0],[0,0,1]],immutable=True)
}

## (VII) Orbits

In [51]:
# Ordered by isotropy, these are all transitive D8-sets
trivial = [[2,1,13,12,0,3,14,15]]
centerC2 = [[24,25,26,27]]
otherC2 = [[4,7,20,23],[22,6,5,21]] # iso4 in GAP
yet_another_C2 = [[19,9,8,18],[10,16,17,11]] # iso8 in GAP

orbit_indices = trivial + centerC2 + otherC2 + yet_another_C2
all_orbit_indices = flatten(orbit_indices)
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(D4_Bitangents_new[b_index])
    list_of_bitangent_orbits.append(orbit)

# remaining_bitangents = rescaled_C3_Bitangents

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(D4_Bitangents_new)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(D4_Bitangents_new[j]))

Orbits of bitangents: [[x + (-0.20871)*y + (-0.9137)*z, x + (-0.20871)*y + 0.9137*z, x + 0.20871*y + 0.9137*I*z, x + 0.20871*y + (-0.9137*I)*z, x + (-4.79129)*y + 4.3778*z, x + (-4.79129)*y + (-4.3778)*z, x + 4.79129*y + (-4.3778*I)*z, x + 4.79129*y + 4.3778*I*z], [x + (-0.58662693)*y, x + (-1.7046609)*y, x + 1.7046609*y, x + 0.58662693*y], [x + y + (-0.59161 - 0.3873*I)*z, x - y + (0.3873 - 0.59161*I)*z, x - y + (-0.3873 + 0.59161*I)*z, x + y + (0.59161 + 0.3873*I)*z], [x + y + (0.59161 - 0.3873*I)*z, x - y + (0.3873 + 0.59161*I)*z, x + y + (-0.59161 + 0.3873*I)*z, x - y + (-0.3873 - 0.59161*I)*z], [x + 1.0*I*y + (-0.86603 + 0.86603*I)*z, x + (-1.0*I)*y + (0.86603 + 0.86603*I)*z, x + 1.0*I*y + (0.86603 - 0.86603*I)*z, x + (-1.0*I)*y + (-0.86603 - 0.86603*I)*z], [x + (-1.0*I)*y + (-1.32288 + 1.32288*I)*z, x + 1.0*I*y + (1.32288 + 1.32288*I)*z, x + (-1.0*I)*y + (1.32288 - 1.32288*I)*z, x + 1.0*I*y + (-1.32288 - 1.32288*I)*z]]

Remaining bitangents:



In [52]:
investigate_orbit(D4_Bitangents_new[24],D8_dict)

x + 1.7046609*y
x + (-0.58662693)*y
x + 0.58662693*y
x + (-1.7046609)*y
I think isotropy is generated by:

r2


# Type VIII: $C_6$<a name="C6"></a>
Doing a new equation the second time around:

In [53]:
F=QQ; T.<x,y,z>=PolynomialRing(F)
C6_Quartic = z^3*y +x^4 -3*x^2*y^2 +y^4
C6_Bitangents = compute_bitangents(C6_Quartic);C6_Bitangents
C6_Bitangents = [normalize(bitangent) for bitangent in C6_Bitangents];C6_Bitangents

The quartic has 28 bitangets, stored in 'Bitangents', and 4 real bitangents, stored in 'RealBitangents'.


[1.0000000*y + (-0.92831777)*z,
 (1.0000000 - 4.6566129e-10*I)*y + (0.46415888 + 0.80394677*I)*z,
 (1.0000000 + 4.6566129e-10*I)*y + (0.46415888 - 0.80394677*I)*z,
 x + 1.85047*y + (-0.39283)*z,
 x + (-1.85047)*y + 0.39283*z,
 x + 0.56125*I*y + 1.18243*I*z,
 x + (-0.56125*I)*y + (-1.18243*I)*z,
 x + (0.70756 - 0.23507*I)*y + (-0.52306 - 0.12198*I)*z,
 x + (-0.70756 + 0.23507*I)*y + (0.52306 + 0.12198*I)*z,
 x + (0.70756 + 0.23507*I)*y + (-0.52306 + 0.12198*I)*z,
 x + (-0.70756 - 0.23507*I)*y + (0.52306 - 0.12198*I)*z,
 x + (-0.56125*I)*y + (-1.02401 + 0.59121*I)*z,
 x + 0.56125*I*y + (1.02401 - 0.59121*I)*z,
 x + 0.56125*I*y + (-1.02401 - 0.59121*I)*z,
 x + (-0.56125*I)*y + (1.02401 + 0.59121*I)*z,
 x + (-0.70756 - 0.23507*I)*y + (-0.15589 + 0.51397*I)*z,
 x + (0.70756 + 0.23507*I)*y + (0.15589 - 0.51397*I)*z,
 x + (-0.70756 + 0.23507*I)*y + (-0.15589 - 0.51397*I)*z,
 x + (0.70756 - 0.23507*I)*y + (0.15589 + 0.51397*I)*z,
 x + (-0.70756 + 0.23507*I)*y + (-0.36717 + 0.39199*I)*z,
 x + (

There are some rounding errors so we edit this set by hand:

In [54]:
C6_Bitangents=[y + (-0.92831777)*z,
 y + (0.46415888 + 0.80394677*I)*z,
 y + (0.46415888 - 0.80394677*I)*z,
 x + 1.85047*y + (-0.39283)*z,
 x + (-1.85047)*y + 0.39283*z,
 x + 0.56125*I*y + 1.18243*I*z,
 x + (-0.56125*I)*y + (-1.18243*I)*z,
 x + (0.70756 - 0.23507*I)*y + (-0.52306 - 0.12198*I)*z,
 x + (-0.70756 + 0.23507*I)*y + (0.52306 + 0.12198*I)*z,
 x + (0.70756 + 0.23507*I)*y + (-0.52306 + 0.12198*I)*z,
 x + (-0.70756 - 0.23507*I)*y + (0.52306 - 0.12198*I)*z,
 x + (-0.56125*I)*y + (-1.02401 + 0.59121*I)*z,
 x + 0.56125*I*y + (1.02401 - 0.59121*I)*z,
 x + 0.56125*I*y + (-1.02401 - 0.59121*I)*z,
 x + (-0.56125*I)*y + (1.02401 + 0.59121*I)*z,
 x + (-0.70756 - 0.23507*I)*y + (-0.15589 + 0.51397*I)*z,
 x + (0.70756 + 0.23507*I)*y + (0.15589 - 0.51397*I)*z,
 x + (-0.70756 + 0.23507*I)*y + (-0.15589 - 0.51397*I)*z,
 x + (0.70756 - 0.23507*I)*y + (0.15589 + 0.51397*I)*z,
 x + (-0.70756 + 0.23507*I)*y + (-0.36717 + 0.39199*I)*z,
 x + (0.70756 - 0.23507*I)*y + (0.36717 - 0.39199*I)*z,
 x + (-0.70756 - 0.23507*I)*y + (-0.36717 - 0.39199*I)*z,
 x + (0.70756 + 0.23507*I)*y + (0.36717 + 0.39199*I)*z,
 x + (-1.85047)*y + (-0.19641 - 0.3402*I)*z,
 x + 1.85047*y + (0.19641 + 0.3402*I)*z,
 x + (-1.85047)*y + (-0.19641 + 0.3402*I)*z,
 x + 1.85047*y + (0.19641 - 0.3402*I)*z,
 y]

In [55]:
# C6 = <r> so the elements are ordered so that dict['ri'] = r^i
C6_elts = [
    [[1,0,0],[0,1,0],[0,0,1]],
    [ [ -1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, E(3) ] ],
    [ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, E(3)^2 ] ],
    [ [ -1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ] ],
    [ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, E(3) ] ],
    [ [ -1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, E(3)^2 ] ]
]


keys = ['r' + str(j) for j in range(0,6)]
C6_dict = {}
for j in range(0,6):
    C6_dict.update({keys[j]:matrix(CC,C6_elts[j],immutable=True)});

## (VIII) Orbits:

In [56]:
e_isotropy = [[24,26,3,25,4,23],[6,11,14,13,12,5],[7,20,18,19,8,17],[16,9,10,22,15,21]]
C2_isotropy = [[0,1,2]]

orbit_indices = e_isotropy + C2_isotropy
all_orbit_indices = flatten(orbit_indices)
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(C6_Bitangents[b_index])
    list_of_bitangent_orbits.append(orbit)

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(C6_Bitangents)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(C6_Bitangents[j]))

Orbits of bitangents: [[x + 1.85047000000000*y + (0.196410000000000 + 0.340200000000000*I)*z, x + 1.85047000000000*y + (0.196410000000000 - 0.340200000000000*I)*z, x + 1.85047000000000*y - 0.392830000000000*z, x + (-1.85047000000000)*y + (-0.196410000000000 + 0.340200000000000*I)*z, x - 1.85047000000000*y + 0.392830000000000*z, x + (-1.85047000000000)*y + (-0.196410000000000 - 0.340200000000000*I)*z], [x + (-0.561250000000000*I)*y + (-1.18243000000000*I)*z, x + (-0.561250000000000*I)*y + (-1.02401000000000 + 0.591210000000000*I)*z, x + (-0.561250000000000*I)*y + (1.02401000000000 + 0.591210000000000*I)*z, x + 0.561250000000000*I*y + (-1.02401000000000 - 0.591210000000000*I)*z, x + 0.561250000000000*I*y + (1.02401000000000 - 0.591210000000000*I)*z, x + 0.561250000000000*I*y + 1.18243000000000*I*z], [x + (0.707560000000000 - 0.235070000000000*I)*y + (-0.523060000000000 - 0.121980000000000*I)*z, x + (0.707560000000000 - 0.235070000000000*I)*y + (0.367170000000000 - 0.391990000000000*I)*z,

In [57]:
investigate_orbit(C6_Bitangents[27],C6_dict)

y
I think isotropy is generated by:

r0
r1
r2
r3
r4
r5


# Type IX: The $S_3$ quartic<a name="S3"></a>
We have an $S_3$-symmetric quartic with equation
$$X^3Z + Y^3 Z + X^2 Y^2 + aXYZ^2 + bZ^4$$
and symmetries ...

## (IX) Old

In [58]:
S3_Quartic = x^3 * z + y^3 * z + x^2 * y^2 - x*y*z^2 + z^4
S3_Bitangents = compute_bitangents(S3_Quartic)
S3_Bitangents = [normalize(bitangent) for bitangent in S3_Bitangents]

rho = exp(2*pi*i/3)

cycle1 = matrix(CC,[[rho,0,0],[0,rho^2,0],[0,0,1]])
transp1 = matrix([[0,1,0],[1,0,0],[0,0,1]])
transp2 = cycle1*transp1
transp3 = cycle1*cycle1*transp1
cycle2 = cycle1*cycle1

The quartic has 28 bitangets, stored in 'Bitangents', and 4 real bitangents, stored in 'RealBitangents'.


In [59]:
orbit1 = [[2]]
orbit2 = []
orbit3 = [[0,26,27],[1,18,19],[3,8,9]]
orbit6 = [[4,5,16,23,22,17],[6,14,21,13,24,11],[7,15,12,20,10,25]]
orbit_indices = orbit1 + orbit2 + orbit3 + orbit6
all_orbit_indices = flatten(orbit_indices)
# Get the running list of orbits that we have
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(S3_Bitangents[b_index])
    list_of_bitangent_orbits.append(orbit)

# remaining_bitangents = rescaled_C3_Bitangents

# print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
# print('Remaining bitangents:\n')
# for j in range(0,len(S3_Bitangents)):
#     if not j in all_orbit_indices:
#         print(str(j) + ': ' + str(S3_Bitangents[j]))

So we get orbits:
$$[S_3/S_3] + 3[S_3/C_2] + 3[S_3/e]$$

## (IX) New

In [60]:
# S3_Quartic_new = x^3 * z + y^3 * z + x^2 * y^2 - 25*x*y*z^2 + 10*z^4
R.<x,y> = PolynomialRing(RR)
maxval=10
S3_Quartic_new_affine = x^3 + y^3 + x^2 * y^2 - 25*x*y + 10
implicit_plot(S3_Quartic_new_affine,(-maxval,maxval),(-maxval,maxval),color='black')

Graphics object consisting of 1 graphics primitive

In [61]:
F=QQ; T.<x,y,z>=PolynomialRing(F)
S3_Quartic_new = x^3 * z + y^3 * z + x^2 * y^2 - 25*x*y*z^2 + 10*z^4
S3_Bitangents = compute_bitangents(S3_Quartic_new)
S3_Bitangents = [normalize(bitangent) for bitangent in S3_Bitangents];S3_Bitangents

The quartic has 28 bitangets, stored in 'Bitangents', and 8 real bitangents, stored in 'RealBitangents'.


[x + 5.98604*y + (-3.59694)*z,
 x + 0.16706*y + (-0.60089)*z,
 x + 21.33127*y + (-111.38015)*z,
 x + y + (-8.59384)*z,
 x + 0.04688*y + (-5.22145)*z,
 z,
 x + 0.28062*y + 0.59766*z,
 x + 3.56351*y + 2.12978*z,
 x + (-1.78175 - 3.08609*I)*y + (-1.06489 + 1.84445*I)*z,
 x + (-1.78175 + 3.08609*I)*y + (-1.06489 - 1.84445*I)*z,
 x + (-0.14031 - 0.24303*I)*y + (-0.29883 + 0.51759*I)*z,
 x + (-0.14031 + 0.24303*I)*y + (-0.29883 - 0.51759*I)*z,
 x + (-0.5 - 0.86603*I)*y + (0.69826 + 4.06576*I)*z,
 x + (-0.5 + 0.86603*I)*y + (0.69826 - 4.06576*I)*z,
 x + (-0.02344 + 0.0406*I)*y + (2.61072 + 4.52191*I)*z,
 x + (-0.02344 - 0.0406*I)*y + (2.61072 - 4.52191*I)*z,
 x + (-0.5 - 0.86603*I)*y + (-3.87019 + 1.42817*I)*z,
 x + (-0.5 + 0.86603*I)*y + (-3.87019 - 1.42817*I)*z,
 x + (-0.5 + 0.86603*I)*y + (4.29692 + 7.44249*I)*z,
 x + (-0.5 - 0.86603*I)*y + (4.29692 - 7.44249*I)*z,
 x + (-10.66564 + 18.47343*I)*y + (55.69008 + 96.45804*I)*z,
 x + (-10.66564 - 18.47343*I)*y + (55.69008 - 96.45804*I)*z,
 x +

## (IX) Dictionary

In [62]:
S3_elts=[ [ [ 0, 1, 0 ], [ 1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, E(3)^2, 0 ], [ E(3), 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, E(3), 0 ], [ E(3)^2, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ] ],
  [ [ E(3)^2, 0, 0 ], [ 0, E(3), 0 ], [ 0, 0, 1 ] ],
  [ [ E(3), 0, 0 ], [ 0, E(3)^2, 0 ], [ 0, 0, 1 ] ] ];

keys = ['r' + str(j) for j in range(0,6)]
S3_dict = {}
for j in range(0,6):
    S3_dict.update({keys[j]:matrix(CC,S3_elts[j],immutable=True)});

## (IX) Orbits

In [63]:
triv_orbit = [[1,23,22,26,27,0],[14,21,15,20,2,4],[6,10,11,8,7,9]]
C2_orbit = [[18,19,3],[12,24,17],[16,25,13]]
S3_orbit = [[5]]

orbit_indices = triv_orbit + C2_orbit
all_orbit_indices = flatten(orbit_indices)
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(C6_Bitangents[b_index])
    list_of_bitangent_orbits.append(orbit)

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(S3_Bitangents)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(S3_Bitangents[j]))

Orbits of bitangents: [[y + (0.464158880000000 + 0.803946770000000*I)*z, x + (-1.85047000000000)*y + (-0.196410000000000 - 0.340200000000000*I)*z, x + (0.707560000000000 + 0.235070000000000*I)*y + (0.367170000000000 + 0.391990000000000*I)*z, x + 1.85047000000000*y + (0.196410000000000 - 0.340200000000000*I)*z, y, y - 0.928317770000000*z], [x + (-0.561250000000000*I)*y + (1.02401000000000 + 0.591210000000000*I)*z, x + (-0.707560000000000 - 0.235070000000000*I)*y + (-0.367170000000000 - 0.391990000000000*I)*z, x + (-0.707560000000000 - 0.235070000000000*I)*y + (-0.155890000000000 + 0.513970000000000*I)*z, x + (0.707560000000000 - 0.235070000000000*I)*y + (0.367170000000000 - 0.391990000000000*I)*z, y + (0.464158880000000 - 0.803946770000000*I)*z, x - 1.85047000000000*y + 0.392830000000000*z], [x + (-0.561250000000000*I)*y + (-1.18243000000000*I)*z, x + (-0.707560000000000 - 0.235070000000000*I)*y + (0.523060000000000 - 0.121980000000000*I)*z, x + (-0.561250000000000*I)*y + (-1.0240100000

In [64]:
investigate_orbit(S3_Bitangents[13],S3_dict)

x + (-0.5 - 0.86602*I)*y + (-3.87017 + 1.42815*I)*z
x + (-0.5 - 0.86603*I)*y + (-3.87018 + 1.42817*I)*z
x + y + (3.1719 + 2.63759*I)*z
x + y + (3.17192 + 2.63759*I)*z
x + (-0.5 + 0.86602*I)*y + (0.69827 - 4.06574*I)*z
x + (-0.5 + 0.86603*I)*y + (0.69826 - 4.06576*I)*z
I think isotropy is generated by:

r3


# Type X: The $K_4$ quartic<a name="K4"></a>
We now look at a quartic with an action by the Klein four group, given by the equation $$X^4 + Y^4 + Z^4 + aX^2 Y^2 + bY^2Z^2 + c Z^2 X^2$$ with action given by $$\text{diag}[-1,1,1] \qquad \text{and}\qquad \text{diag}[1,-1,1]$$

## Old

In [65]:
K4_Quartic = x^4 + y^4 + z^4 + x^2*y^2 + y^2*z^2 + z^2*x^2
K4_Bitangents = compute_bitangents(K4_Quartic)
K4_Bitangents = [normalize(bitangent) for bitangent in K4_Bitangents]

# There are three subgroups of K4, the left, middle, and diagonal copies of C2
left_generator = matrix([[-1,0,0],[0,1,0],[0,0,1]])
right_generator = matrix([[1,0,0],[0,-1,0],[0,0,1]])
diagonal_generator = matrix([[-1,0,0],[0,-1,0],[0,0,1]])

# Make different lists to indicate different isotropy subgroups
left_isotropy_indices = [[0,1],[2,3]]
right_isotropy_indices = [[16,18],[17,19]]
diagonal_isotropy_indices = [[24,26],[25,27]]
remaining_orbits = [[12,13,20,21],[4,5,6,7],[8,9,10,11],[14,15,22,23]]
orbit_indices = left_isotropy_indices + right_isotropy_indices + diagonal_isotropy_indices + remaining_orbits
# Flatten the union
all_orbit_indices = flatten(orbit_indices)

# Get the running list of orbits that we have
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(K4_Bitangents[b_index])
    list_of_bitangent_orbits.append(orbit)

# remaining_bitangents = rescaled_C3_Bitangents

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(K4_Bitangents)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(K4_Bitangents[j]))

The quartic has 28 bitangets, stored in 'Bitangents', and 4 real bitangents, stored in 'RealBitangents'.
Orbits of bitangents: [[y + (-0.57735027 - 0.81649658*I)*z, y + (-0.57735027 + 0.81649658*I)*z], [y + (0.57735027 + 0.81649658*I)*z, y + (0.57735027 - 0.81649658*I)*z], [x + (-0.57735027 - 0.81649658*I)*z, x + (0.57735027 + 0.81649658*I)*z], [x + (-0.57735027 + 0.81649658*I)*z, x + (0.57735027 - 0.81649658*I)*z], [x + (-0.57735027 - 0.81649658*I)*y, x + (0.57735027 + 0.81649658*I)*y], [x + (-0.57735027 + 0.81649658*I)*y, x + (0.57735027 - 0.81649658*I)*y], [x + y - z, x - y + z, x - y - z, x + y + z], [x + (-1.41421*I)*y - z, x + 1.41421*I*y + z, x + 1.41421*I*y - z, x + (-1.41421*I)*y + z], [x + y + 1.41421*I*z, x - y + (-1.41421*I)*z, x - y + 1.41421*I*z, x + y + (-1.41421*I)*z], [x + 0.70711*I*y + (-0.70711*I)*z, x + (-0.70711*I)*y + 0.70711*I*z, x + (-0.70711*I)*y + (-0.70711*I)*z, x + 0.70711*I*y + 0.70711*I*z]]

Remaining bitangents:



In [66]:
candidate_orbit(K4_Bitangents[25],[left_generator,right_generator,diagonal_generator])

x + (-0.57735027 + 0.81649658*I)*y
x + (0.57735027 - 0.81649658*I)*y
x + (0.57735027 - 0.81649658*I)*y
x + (-0.57735027 + 0.81649658*I)*y
I think isotropy is generated by:

[-1  0  0]
[ 0 -1  0]
[ 0  0  1]


So the orbits for bitangents on our $K_4$-equivariant quartic have the form
$$2[K_4/C_2^L] + 2[K_4/C_2^\Delta] + 2[K_4/C_2^R] + 4[K_4/e]$$

## (X) New

In [67]:
F=QQ; T.<x,y,z>=PolynomialRing(F)

K4_Quartic_new = x^4 + y^4 + z^4 -9*x^2*y^2 - 3*y^2*z^2 - 8*x^2*z^2

K4_Bitangents_new = compute_bitangents(K4_Quartic_new)
K4_Bitangents_new = [normalize(bitangent) for bitangent in K4_Bitangents_new];K4_Bitangents_new

The quartic has 28 bitangets, stored in 'Bitangents', and 16 real bitangents, stored in 'RealBitangents'.


[y + (-0.71838335*I)*z,
 y + 0.71838335*I*z,
 y + 1.2287796*I*z,
 y + (-1.2287796*I)*z,
 x + 2.49875*y + (-1.56867)*z,
 x + (-2.49875)*y + 1.56867*z,
 x + 0.65498*y + (-0.63748)*z,
 x + (-0.65498)*y + 0.63748*z,
 x + 0.4002*y + (-0.41118)*z,
 x + (-0.4002)*y + 0.41118*z,
 x + 1.52676*y + (-2.43201)*z,
 x + (-1.52676)*y + 2.43201*z,
 x + (-1.52676)*y + (-2.43201)*z,
 x + 1.52676*y + 2.43201*z,
 x + (-0.4002)*y + (-0.41118)*z,
 x + 0.4002*y + 0.41118*z,
 x + (-0.65498)*y + (-0.63748)*z,
 x + 0.65498*y + 0.63748*z,
 x + (-2.49875)*y + (-1.56867)*z,
 x + 2.49875*y + 1.56867*z,
 1.0000000*x + (-0.24805077*I)*z,
 1.0000000*x + 0.24805077*I*z,
 x + 1.0273042*I*z,
 x + (-1.0273042*I)*z,
 x + (-0.24959123*I)*y,
 x + 0.24959123*I*y,
 x + 1.1565916*I*y,
 x + (-1.1565916*I)*y]

## (X) Dictionary

In [68]:
K4_dict = {}
K4_dict.update({'id':matrix(CC,[[1,0,0],[0,1,0],[0,0,1]],immutable=True)})
K4_dict.update({'leftgen':matrix(CC,[[-1,0,0],[0,1,0],[0,0,1]],immutable=True)})
K4_dict.update({'rightgen':matrix(CC,[[1,0,0],[0,-1,0],[0,0,1]],immutable=True)})
K4_dict.update({'diaggen':matrix(CC,[[-1,0,0],[0,-1,0],[0,0,1]],immutable=True)})

## (X) Orbits

In [69]:
# e_orb = [[4,5,18,19],[6,7,16,17],[8,9,14,15],[10,11,12,13],[20,21,24,25]]
# C2L = [[0,1],[2,3]]
# C2R = [[22,23],[26,27]]

orbit_indices = []
# Flatten the union
all_orbit_indices = flatten(orbit_indices)

# Get the running list of orbits that we have
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(K4_Bitangents_new[b_index])
    list_of_bitangent_orbits.append(orbit)

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(K4_Bitangents_new)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(K4_Bitangents_new[j]))

Orbits of bitangents: []

Remaining bitangents:

0: y + (-0.71838335*I)*z
1: y + 0.71838335*I*z
2: y + 1.2287796*I*z
3: y + (-1.2287796*I)*z
4: x + 2.49875*y + (-1.56867)*z
5: x + (-2.49875)*y + 1.56867*z
6: x + 0.65498*y + (-0.63748)*z
7: x + (-0.65498)*y + 0.63748*z
8: x + 0.4002*y + (-0.41118)*z
9: x + (-0.4002)*y + 0.41118*z
10: x + 1.52676*y + (-2.43201)*z
11: x + (-1.52676)*y + 2.43201*z
12: x + (-1.52676)*y + (-2.43201)*z
13: x + 1.52676*y + 2.43201*z
14: x + (-0.4002)*y + (-0.41118)*z
15: x + 0.4002*y + 0.41118*z
16: x + (-0.65498)*y + (-0.63748)*z
17: x + 0.65498*y + 0.63748*z
18: x + (-2.49875)*y + (-1.56867)*z
19: x + 2.49875*y + 1.56867*z
20: 1.0000000*x + (-0.24805077*I)*z
21: 1.0000000*x + 0.24805077*I*z
22: x + 1.0273042*I*z
23: x + (-1.0273042*I)*z
24: x + (-0.24959123*I)*y
25: x + 0.24959123*I*y
26: x + 1.1565916*I*y
27: x + (-1.1565916*I)*y


In [70]:
investigate_orbit(K4_Bitangents_new[24],K4_dict)

x + (-0.24959123*I)*y
x + 0.24959123*I*y
I think isotropy is generated by:

id
diaggen


# Type XI: The $C_3$ quartic<a name="C3"></a>
The first one we're trying is the one with automorphism group $C_3$ which we will call `C3_Quartic`. The associated list of bitangents is `C3_Bitangents`.

In [71]:
C3_Quartic = z^3*y + x*(x-y)*(x-2*y)*(x-3*y)
C3_Bitangents = compute_bitangents(C3_Quartic)
rescaled_C3_Bitangents = [normalize(bitangent) for bitangent in C3_Bitangents]
# for j in range(0,len(rescaled_C3_Bitangents)):
#     print(str(j) + ': ' + str(rescaled_C3_Bitangents[j]))

The quartic has 28 bitangets, stored in 'Bitangents', and 4 real bitangents, stored in 'RealBitangents'.


In [72]:
# The action generating the C3 orbit
rho = exp(2*pi*i/3)
M = matrix([[1,0,0],[0,1,0],[0,0,rho]])

orbit_indices = [[1,15,16],[2,11,12],[5,14,19],[0,25,26],[3,18,23],[27],[4,17,24],[6,13,20],[7,10,21],[8,9,22]]
all_orbit_indices = flatten(orbit_indices)

# Get the running list of orbits that we have
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(rescaled_C3_Bitangents[b_index])
    list_of_bitangent_orbits.append(orbit)

# remaining_bitangents = rescaled_C3_Bitangents

# gp_action(M,rescaled_C3_Bitangents[22])
print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(rescaled_C3_Bitangents)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(rescaled_C3_Bitangents[j]))

Orbits of bitangents: [[y - z, y + (0.50000000 - 0.86602540*I)*z, y + (0.50000000 + 0.86602540*I)*z], [x + 0.21958*y + (-0.40224)*z, x + 0.21958*y + (0.20112 - 0.34835*I)*z, x + 0.21958*y + (0.20112 + 0.34835*I)*z], [x + (-1.5 - 0.48203*I)*y + (0.98297 + 0.56752*I)*z, x + (-1.5 - 0.48203*I)*y + (-1.13503*I)*z, x + (-1.5 - 0.48203*I)*y + (-0.98297 + 0.56752*I)*z], [x + (-3.21958)*y + 0.40224*z, x + (-3.21958)*y + (-0.20112 + 0.34835*I)*z, x + (-3.21958)*y + (-0.20112 - 0.34835*I)*z], [x + (-2.08999 + 0.20908*I)*y + (0.54475 + 0.13927*I)*z, x + (-2.08999 + 0.20908*I)*y + (-0.15177 - 0.5414*I)*z, x + (-2.08999 + 0.20908*I)*y + (-0.39298 + 0.40213*I)*z], [y], [x + (-2.08999 - 0.20908*I)*y + (0.54475 - 0.13927*I)*z, x + (-2.08999 - 0.20908*I)*y + (-0.15177 + 0.5414*I)*z, x + (-2.08999 - 0.20908*I)*y + (-0.39298 - 0.40213*I)*z], [x + (-1.5 + 0.48203*I)*y + (0.98297 - 0.56752*I)*z, x + (-1.5 + 0.48203*I)*y + 1.13503*I*z, x + (-1.5 + 0.48203*I)*y + (-0.98297 - 0.56752*I)*z], [x + (-0.91001 + 0

Okay so now we know the orbits of the $C_3$-equivariant quartic: $9[C_3/e]+[C_3/C_3]$

# Type XII: The $C_2$ quartic

## (XII) Old

In [73]:
L2 = y^2 - z^2
L4 = y^4 - y^2*z^2 + z^4
C2_Quartic = x^4 + x^2*L2 + L4

C2_Bitangents = compute_bitangents(C2_Quartic);
C2_Bitangents = [normalize(b) for b in C2_Bitangents]

e_orbits = [[0],[1],[2],[3]]
C2_orbits = [[4,5],[6,7],[8,9],[10,11],[12,14],[13,15],[16,17],[18,19],[20,21],[22,23],[24,26],[25,27]]

C2_Bitangents

The quartic has 28 bitangets, stored in 'Bitangents', and 4 real bitangents, stored in 'RealBitangents'.


[y + (-0.81649658 - 0.57735027*I)*z,
 y + (-0.81649658 + 0.57735027*I)*z,
 y + (0.81649658 + 0.57735027*I)*z,
 y + (0.81649658 - 0.57735027*I)*z,
 x + (-1.41421*I)*y + 1.0*I*z,
 x + 1.41421*I*y + (-1.0*I)*z,
 x + y + (-1.41421)*z,
 x - y + 1.41421*z,
 x - y + (-1.41421)*z,
 x + y + 1.41421*z,
 x + 1.41421*I*y + 1.0*I*z,
 x + (-1.41421*I)*y + (-1.0*I)*z,
 x + (-0.81649658 - 0.57735027*I)*z,
 x + (-0.81649658 + 0.57735027*I)*z,
 x + (0.81649658 + 0.57735027*I)*z,
 x + (0.81649658 - 0.57735027*I)*z,
 x + 0.70711*I*y + (-0.70711)*z,
 x + (-0.70711*I)*y + 0.70711*z,
 x + y + 1.0*I*z,
 x - y + (-1.0*I)*z,
 x + (-0.70711*I)*y + (-0.70711)*z,
 x + 0.70711*I*y + 0.70711*z,
 x - y + 1.0*I*z,
 x + y + (-1.0*I)*z,
 x + (-0.57735027 - 0.81649658*I)*y,
 x + (-0.57735027 + 0.81649658*I)*y,
 x + (0.57735027 + 0.81649658*I)*y,
 x + (0.57735027 - 0.81649658*I)*y]

In [74]:
m = matrix(CC,[[-1,0,0],[0,1,0],[0,0,1]])
print(C2_Bitangents[4])
gp_action(m,C2_Bitangents[4])

x + (-1.41421*I)*y + 1.0*I*z


x + 1.41421*I*y + (-1.0*I)*z

## (XII) New

In [75]:
L2_new = -y^2 - 4*z^2
L4_new = y^4 - y^2*z^2 - z^4
C2_Quartic_new = x^4 + x^2*L2_new + L4_new

C2_Bitangents_new = compute_bitangents(C2_Quartic_new);
C2_Bitangents_new = [normalize(b) for b in C2_Bitangents_new]

The quartic has 28 bitangets, stored in 'Bitangents', and 4 real bitangents, stored in 'RealBitangents'.


## (XII) Orbits
The action is given by the diagonal matrix $\begin{pmatrix} -1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1\end{pmatrix}$ so it just flips the sign on the $x$ (or on both $y$ and $z$), so we can eyeball the orbits. Note if the coefficient on $x$ is trivial then the action doesn't affect the bitangent.

In [76]:
e_orbs = []
# [[0],[1],[2],[3]]
C2_orbs = []
# [[4,5],[6,7],[8,9],[10,11],[12,13],[14,15],[16,17],..]
orbit_indices = e_orbs + C2_orbs
all_orbit_indices = flatten(orbit_indices)

# Get the running list of orbits that we have
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(C2_Bitangents_new[b_index])
    list_of_bitangent_orbits.append(orbit)

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(C2_Bitangents_new)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(C2_Bitangents_new[j]))

Orbits of bitangents: []

Remaining bitangents:

0: y + (-2.2947737)*z
1: y + 2.2947737*z
2: y + (-1.1251606*I)*z
3: y + 1.1251606*I*z
4: x + (0.96825 + 0.25*I)*y + (-0.99367 - 0.48721*I)*z
5: x + (-0.96825 - 0.25*I)*y + (0.99367 + 0.48721*I)*z
6: x + (0.96825 - 0.25*I)*y + (-0.99367 + 0.48721*I)*z
7: x + (-0.96825 + 0.25*I)*y + (0.99367 - 0.48721*I)*z
8: x + (-0.79057 - 0.61237*I)*y + (-0.3978 + 0.81133*I)*z
9: x + (0.79057 + 0.61237*I)*y + (0.3978 - 0.81133*I)*z
10: x + (-0.79057 + 0.61237*I)*y + (-0.3978 - 0.81133*I)*z
11: x + (0.79057 - 0.61237*I)*y + (0.3978 + 0.81133*I)*z
12: x + (0.79057 - 0.61237*I)*y + (-0.3978 - 0.81133*I)*z
13: x + (-0.79057 + 0.61237*I)*y + (0.3978 + 0.81133*I)*z
14: x + (0.79057 + 0.61237*I)*y + (-0.3978 + 0.81133*I)*z
15: x + (-0.79057 - 0.61237*I)*y + (0.3978 - 0.81133*I)*z
16: x + (-0.96825 - 0.25*I)*y + (-0.99367 - 0.48721*I)*z
17: x + (0.96825 + 0.25*I)*y + (0.99367 + 0.48721*I)*z
18: x + (-0.96825 + 0.25*I)*y + (-0.99367 + 0.48721*I)*z
19: x + (0.968

The action of $m$ leaves the bitangent stable if there's no $x$ coefficient, otherwise it swaps the sign on the $y$ and $z$ coefficients.

In [77]:
2+2

4

# Trott / Blum-Guinand

In [78]:
Trott = 144*(x^4+y^4) - 255*(x^2+y^2)*z^2 + 350*x^2*y^2+81*z^4
Trott_bis = compute_bitangents(Trott);Trott_bis


The quartic has 28 bitangets, stored in 'Bitangents', and 4 real bitangents, stored in 'RealBitangents'.


[(-1.0789206623656498 + 0.5507999555567185*I)*y + z,
 (-1.0789206623656498 - 0.5507999555567185*I)*y + z,
 (1.0789206623656498 - 0.5507999555567185*I)*y + z,
 (1.0789206623656498 + 0.5507999555567185*I)*y + z,
 (-1.0534120923965482*I)*x + (-2.0634498260329526)*y + z,
 1.0534120923965482*I*x + (-2.0634498260329526)*y + z,
 (-0.6461670724975701)*x + (-0.6461670724975701)*y + z,
 0.6461670724975701*x + (-0.6461670724975701)*y + z,
 (-0.6461670724975701)*x + 0.6461670724975701*y + z,
 0.6461670724975701*x + 0.6461670724975701*y + z,
 (-1.0534120923965482*I)*x + 2.0634498260329526*y + z,
 1.0534120923965482*I*x + 2.0634498260329526*y + z,
 (-2.0634498260329526)*x + 1.0534120923965482*I*y + z,
 2.0634498260329526*x + 1.0534120923965482*I*y + z,
 (-2.0634498260329526)*x + (-1.0534120923965482*I)*y + z,
 2.0634498260329526*x + (-1.0534120923965482*I)*y + z,
 (-1.2657281447187063*I)*x + (-1.2657281447187063*I)*y + z,
 1.2657281447187063*I*x + (-1.2657281447187063*I)*y + z,
 (-1.2657281447187063

*Weird*: Why are some of them complex?

# Edge quartic

In [79]:
Edge = 25*(x^4 + y^4 + z^4) -34*(x^2*y^2 + x^2*z^2 + y^2*z^2)
Edge_bis = compute_bitangents(Edge);
Edge_bis = [normalize(b) for b in Edge_bis];Edge_bis

The quartic has 28 bitangets, stored in 'Bitangents', and 28 real bitangents, stored in 'RealBitangents'.


[y + (-0.50000000)*z,
 y + (-2.0000000)*z,
 y + 2.0000000*z,
 y + 0.50000000*z,
 x + y + (-0.6)*z,
 x - y + 0.6*z,
 x + 0.6*y - z,
 x + (-0.6)*y + z,
 x + (-0.6)*y - z,
 x + 0.6*y + z,
 x - y + (-0.6)*z,
 x + y + 0.6*z,
 x + y - z,
 x + 1.66667*y + (-1.66667)*z,
 x + (-1.66667)*y + 1.66667*z,
 x - y + z,
 x + (-0.50000000)*z,
 x + (-2.0000000)*z,
 x + 2.0000000*z,
 x + 0.50000000*z,
 x - y - z,
 x + (-1.66667)*y + (-1.66667)*z,
 x + 1.66667*y + 1.66667*z,
 x + y + z,
 x + (-0.50000000)*y,
 x + (-2.0000000)*y,
 x + 2.0000000*y,
 x + 0.50000000*y]

In [80]:
c2r1 = [[16,19,0,3],[1,18,2,17]]
c2r5 = [[11,10,5,4],[20,15,23,12]]
e_istrop = [[21,13,7,9,14,22,6,8]]
c2_r0 = [[24,25,26,27]]

orbit_indices = c2r1 + c2r5 + e_istrop

all_orbit_indices = flatten(orbit_indices)

# Get the running list of orbits that we have
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(Edge_bis[b_index])
    list_of_bitangent_orbits.append(orbit)

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(Edge_bis)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(Edge_bis[j]))

Orbits of bitangents: [[x + (-0.50000000)*z, x + 0.50000000*z, y + (-0.50000000)*z, y + 0.50000000*z], [y + (-2.0000000)*z, x + 2.0000000*z, y + 2.0000000*z, x + (-2.0000000)*z], [x + y + 0.6*z, x - y + (-0.6)*z, x - y + 0.6*z, x + y + (-0.6)*z], [x - y - z, x - y + z, x + y + z, x + y - z], [x + (-1.66667)*y + (-1.66667)*z, x + 1.66667*y + (-1.66667)*z, x + (-0.6)*y + z, x + 0.6*y + z, x + (-1.66667)*y + 1.66667*z, x + 1.66667*y + 1.66667*z, x + 0.6*y - z, x + (-0.6)*y - z]]

Remaining bitangents:

24: x + (-0.50000000)*y
25: x + (-2.0000000)*y
26: x + 2.0000000*y
27: x + 0.50000000*y


## Edge with $S_4$

In [81]:
TypeIV_S4 = [ [ [ -1, 0, 0 ], [ 0, -1, 0 ], [ 0, 0, 1 ] ],
  [ [ -1, 0, 0 ], [ 0, 0, -1 ], [ 0, -1, 0 ] ],
  [ [ -1, 0, 0 ], [ 0, 0, 1 ], [ 0, 1, 0 ] ],
  [ [ -1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, -1 ] ],
  [ [ 0, -1, 0 ], [ -1, 0, 0 ], [ 0, 0, -1 ] ],
  [ [ 0, -1, 0 ], [ 0, 0, -1 ], [ 1, 0, 0 ] ],
  [ [ 0, -1, 0 ], [ 0, 0, 1 ], [ -1, 0, 0 ] ],
  [ [ 0, -1, 0 ], [ 1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, 0, -1 ], [ -1, 0, 0 ], [ 0, 1, 0 ] ],
  [ [ 0, 0, -1 ], [ 0, -1, 0 ], [ -1, 0, 0 ] ],
  [ [ 0, 0, -1 ], [ 0, 1, 0 ], [ 1, 0, 0 ] ],
  [ [ 0, 0, -1 ], [ 1, 0, 0 ], [ 0, -1, 0 ] ],
  [ [ 0, 0, 1 ], [ -1, 0, 0 ], [ 0, -1, 0 ] ],
  [ [ 0, 0, 1 ], [ 0, -1, 0 ], [ 1, 0, 0 ] ],
  [ [ 0, 0, 1 ], [ 0, 1, 0 ], [ -1, 0, 0 ] ],
  [ [ 0, 0, 1 ], [ 1, 0, 0 ], [ 0, 1, 0 ] ],
  [ [ 0, 1, 0 ], [ -1, 0, 0 ], [ 0, 0, 1 ] ],
  [ [ 0, 1, 0 ], [ 0, 0, -1 ], [ -1, 0, 0 ] ],
  [ [ 0, 1, 0 ], [ 0, 0, 1 ], [ 1, 0, 0 ] ],
  [ [ 0, 1, 0 ], [ 1, 0, 0 ], [ 0, 0, -1 ] ],
  [ [ 1, 0, 0 ], [ 0, -1, 0 ], [ 0, 0, -1 ] ],
  [ [ 1, 0, 0 ], [ 0, 0, -1 ], [ 0, 1, 0 ] ],
  [ [ 1, 0, 0 ], [ 0, 0, 1 ], [ 0, -1, 0 ] ],
  [ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ] ] ]
Edge_S4_dict = {}
keys = ['r' + str(j) for j in range(0,24)]
for j in range(0,24):
    Edge_S4_dict.update({keys[j]:matrix(CC,TypeIV_S4[j],immutable=True)});

In [82]:
c2r20 = [[18,25,24,27,16,19,26,17,0,1,2,3]]
c2r4 = [[21,14,13,22,11,4,10,5,7,8,9,6]]

orbit_indices = c2r20 + c2r4

all_orbit_indices = flatten(orbit_indices)

# Get the running list of orbits that we have
list_of_bitangent_orbits = []
for o in orbit_indices:
    orbit = []
    for b_index in o:
        orbit.append(Edge_bis[b_index])
    list_of_bitangent_orbits.append(orbit)

print('Orbits of bitangents: ' + str(list_of_bitangent_orbits) + '\n')
print('Remaining bitangents:\n')
for j in range(0,len(Edge_bis)):
    if not j in all_orbit_indices:
        print(str(j) + ': ' + str(Edge_bis[j]))

Orbits of bitangents: [[x + 2.0000000*z, x + (-2.0000000)*y, x + (-0.50000000)*y, x + 0.50000000*y, x + (-0.50000000)*z, x + 0.50000000*z, x + 2.0000000*y, x + (-2.0000000)*z, y + (-0.50000000)*z, y + (-2.0000000)*z, y + 2.0000000*z, y + 0.50000000*z], [x + (-1.66667)*y + (-1.66667)*z, x + (-1.66667)*y + 1.66667*z, x + 1.66667*y + (-1.66667)*z, x + 1.66667*y + 1.66667*z, x + y + 0.6*z, x + y + (-0.6)*z, x - y + (-0.6)*z, x - y + 0.6*z, x + (-0.6)*y + z, x + (-0.6)*y - z, x + 0.6*y + z, x + 0.6*y - z]]

Remaining bitangents:

12: x + y - z
15: x - y + z
20: x - y - z
23: x + y + z


In [83]:
investigate_orbit(Edge_bis[12],Edge_S4_dict)

x + y + z
x - y + z
x - y - z
x + y - z
I think isotropy is generated by:

r2
r4
r11
r13
r17
r23


In [84]:
Edge_S4_dict['r4']

[0.000000000000000 -1.00000000000000 0.000000000000000]
[-1.00000000000000 0.000000000000000 0.000000000000000]
[0.000000000000000 0.000000000000000 -1.00000000000000]